In [ ]:
%matplotlib qt
import os
import os.path as op
import numpy as np
from tqdm.notebook import tqdm
from tqdm.contrib import tzip
from tqdm.contrib.itertools import product
import itertools
import pickle
####
import mne
# from mne.preprocessing import create_ecg_epochs, create_eog_epochs
# from mne.coreg import Coregistration
# from mne.minimum_norm import make_inverse_operator, apply_inverse_raw
import learn_graph
# from mne_connectivity import spectral_connectivity_epochs, spectral_connectivity_time
####
import seaborn as sns
import matplotlib.pyplot as plt
# from mpl_toolkits.axes_grid1 import make_axes_locatable
# import nilearn
from nilearn.plotting import view_connectome, plot_connectome
####
# import torch
# from umap.umap_ import UMAP
# from sklearn.cluster import DBSCAN, KMeans
# from matplotlib.animation import FuncAnimation
####
from scipy.stats import ttest_ind, mannwhitneyu, shapiro
import statsmodels.api as sm
# from sklearn.mixture import GaussianMixture
# from sklearn.decomposition import PCA
####
# from tensorpac import Pac
from scipy.signal import coherence

Loading the recordings

In [ ]:
# create a dictionary of subjects with their files
directory = '/Users/payamsadeghishabestari/KI_MEG/KI_RS/tinmeg3' 
folders_dict = {}
for folder in sorted(os.listdir(directory)): ## iterate over folders in that directory
    f = os.path.join(directory, folder)
    if os.path.isdir(f): ## select only folders
        folders_dict[folder] = f

raw_rs_dict = {}
raw_er_dict = {}
subjects = list(folders_dict.keys())
folders = list(folders_dict.values())
for subject_id, folder in zip(subjects, folders):
    raw_er_dict[subject_id] = []
    files = sorted(os.listdir(folder))
    if '.DS_Store' in files:
        files.remove('.DS_Store')
    for file in files:
        if 'empty_room_before' in file:
            raw_er_dict[subject_id].append(os.path.join(folder, files[1]))
        if 'empty_room_after' in file:
            raw_er_dict[subject_id].append(os.path.join(folder, files[0]))
    f_rs = os.path.join(folder, files[-1])
    raw_rs_dict[subject_id] = f_rs

Preprocessing and processing of recordings

In [ ]:
## setting up parameters
verbose = False
sfreq = 250
(l_freq, h_freq) = (0.1, 80)
(l_freq_delta, h_freq_delta) = (0.5, 4)
(l_freq_theta, h_freq_theta) = (4, 8)
(l_freq_alpha, h_freq_alpha) = (8, 13)
(l_freq_beta, h_freq_beta) = (13, 30)
(l_freq_gamma, h_freq_gamma) = (30, 45)
subjects_dir = '/Applications/freesurfer/7.4.1/subjects'
fname_fsaverage_src = '/Users/payamsadeghishabestari/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif'
src_to = mne.read_source_spaces(fname_fsaverage_src, verbose=False)
method = "dSPM"
snr = 3.0
lambda2 = 1.0 / snr**2
atlases = ['aparc', 'aparc.a2009s']
modes = ['mean', 'mean_flip']

In [ ]:
for subject in tqdm(subjects[16:]): 
    
    ## reading the resting state and empty room recordings and computing noise covariance
    start_time = time.time()
    print(f'reading the MEG file of subejct {subject} ...')
    fname = raw_rs_dict[subject]
    raw = mne.io.read_raw_fif(fname=fname, preload=True, allow_maxshield=True, verbose=verbose)
    raw_er_fnames = raw_er_dict[subject]
    print(f'reading and computing noise covariance of the subject {subject} ...')
    if len(raw_er_fnames) == 0:
        noise_cov = mne.make_ad_hoc_cov(info=raw.info, std=None, verbose=verbose) # 5 fT/cm, 20 fT
    if len(raw_er_fnames) == 1:
        raw_er = mne.io.read_raw_fif(fname=raw_er_fnames[0], preload=True,
                                    allow_maxshield=True, verbose=verbose)
        noise_cov = mne.compute_raw_covariance(raw=raw_er, method='empirical', verbose=verbose)
    if len(raw_er_fnames) == 2:   
        raw_er_1 = mne.io.read_raw_fif(fname=raw_er_fnames[0], preload=True,
                                    allow_maxshield=True, verbose=verbose)
        raw_er_2 = mne.io.read_raw_fif(fname=raw_er_fnames[1], preload=True,
                                    allow_maxshield=True, verbose=verbose)
        raw_er = mne.concatenate_raws(raws=[raw_er_1, raw_er_2], verbose=verbose)
        noise_cov = mne.compute_raw_covariance(raw=raw_er, method='empirical', verbose=verbose)    
    
    # ###### Sensor Space ######

    ## resampling and filtering the data
    print('resampling and filtering the data, be patient, will last a while ...')
    raw.resample(sfreq=sfreq, verbose=verbose)
    raw.filter(l_freq=l_freq, h_freq=h_freq, verbose=verbose) 

    ## creating ECG and EOG evoked responses
    ecg_evoked_meg,  ecg_evoked_grad = create_ecg_epochs(raw,
                                    verbose=verbose).average().apply_baseline(baseline=(None, -0.2),
                                    verbose=verbose).plot_joint(picks=['meg', 'grad'], show=False)
    eog_evoked_meg,  eog_evoked_grad = create_eog_epochs(raw,
                                    verbose=verbose).average().apply_baseline(baseline=(None, -0.2),
                                    verbose=verbose).plot_joint(picks=['meg', 'grad'], show=False)

    ## computing ICA and remove ECG, EOG and muscle artifacts (if any) and interpolating (if any)
    print('computing ICA (this might take a while) ...')
    ica = mne.preprocessing.ICA(n_components=0.95, max_iter=800, method='infomax',
                                random_state=42, fit_params=dict(extended=True)) 
    ica.fit(raw, verbose=verbose) 
    ecg_indices, ecg_scores = ica.find_bads_ecg(raw, method="ctps", measure='zscore', verbose=verbose)
    if len(ecg_indices) > 0:
        ecg_component = ica.plot_properties(raw, picks=ecg_indices, verbose=verbose, show=False)
    emg_indices, emg_scores = ica.find_bads_muscle(raw, verbose=verbose)
    if len(emg_indices) > 0:
        emg_component = ica.plot_properties(raw, picks=emg_indices, verbose=verbose, show=False)
    eog_indices, eog_scores = ica.find_bads_eog(raw, ch_name='EOG002', verbose=verbose) 
    if len(eog_indices) > 0:
        eog_component = ica.plot_properties(raw, picks=eog_indices, verbose=verbose, show=False)
    saccade_indices, saccade_scores = ica.find_bads_eog(raw, ch_name='EOG001', verbose=verbose) 
    if len(saccade_indices) > 0:
        saccade_component = ica.plot_properties(raw, picks=saccade_indices, verbose=verbose, show=False)

    exclude_idxs = ecg_indices + emg_indices + eog_indices + saccade_indices
    ica.apply(raw, exclude=exclude_idxs, verbose=verbose)
    raw.interpolate_bads(verbose=verbose)

    ## bandpassing the recording into alpha and gamma bands and saving them
    print('Bandpassing data ...')
    raw_delta = raw.copy().filter(l_freq=l_freq_delta, h_freq=h_freq_delta, verbose=verbose)
    raw_theta = raw.copy().filter(l_freq=l_freq_theta, h_freq=h_freq_theta, verbose=verbose)
    raw_beta = raw.copy().filter(l_freq=l_freq_beta, h_freq=h_freq_beta, verbose=verbose)

    fname_delta = f'/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg3/delta/{subject}_raw_tsss.fif'
    fname_theta = f'/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg3/theta/{subject}_raw_tsss.fif'
    fname_beta = f'/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg3/beta/{subject}_raw_tsss.fif'
    raw_delta.save(fname=fname_delta, overwrite=True, verbose=verbose)
    raw_theta.save(fname=fname_theta, overwrite=True, verbose=verbose)
    raw_beta.save(fname=fname_beta, overwrite=True, verbose=verbose)


    # fname_alpha = f'/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg3/alpha/{subject}_raw_tsss.fif'
    # fname_gamma = f'/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg3/gamma/{subject}_raw_tsss.fif' 
    # raw_alpha = mne.io.read_raw(fname=fname_alpha, preload=True, verbose=False)
    # raw_gamma = mne.io.read_raw(fname=fname_gamma, preload=True, verbose=False)
    
    # fname_delta = f'/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg1/delta/{subject}_raw_tsss.fif'
    # fname_theta = f'/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg1/theta/{subject}_raw_tsss.fif'
    # fname_beta = f'/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg1/beta/{subject}_raw_tsss.fif'
    # raw_delta = mne.io.read_raw(fname=fname_delta, preload=True, verbose=False)
    # raw_theta = mne.io.read_raw(fname=fname_theta, preload=True, verbose=False)
    # raw_beta = mne.io.read_raw(fname=fname_beta, preload=True, verbose=False)

    ###### Source Space ######
    spacing = "oct6"
    if subject == "1004":
        spacing = "oct4"
    if subject == "1045":
        spacing = "ico5"
    
    ## setting up the surface source space
    print(f'Setting up bilateral hemisphere surface-based source space with subsampling for subject {subject} ...')
    src = mne.setup_source_space(subject=f'{subject}', spacing=spacing,
                                subjects_dir=subjects_dir, n_jobs=-1, verbose=True)

    ## setting up the boundary-element model (BEM) 
    print(f'Creating a BEM model for the subject ...')
    bem_model = mne.make_bem_model(subject=f'{subject}', ico=4, subjects_dir=subjects_dir, verbose=True)  
    bem = mne.make_bem_solution(bem_model, verbose=verbose)
    
    ## aligning coordinate frame (coregistration MEG-MRI)
    print(f'Coregistering MRI with a subjects head shape ...')
    coreg = Coregistration(raw_delta.info, f'{subject}', subjects_dir, fiducials='auto')
    coreg.fit_fiducials(verbose=verbose)
    coreg.fit_icp(n_iterations=40, nasion_weight=2.0, verbose=verbose) 
    coreg.omit_head_shape_points(distance=5.0 / 1000)
    coreg.fit_icp(n_iterations=40, nasion_weight=10, verbose=verbose) 
    fname_trans = f'/Users/payamsadeghishabestari/meg_gsp/trans/{subject}-trans.fif'
    mne.write_trans(fname_trans, coreg.trans, overwrite=True, verbose=verbose)
    
    ## computing the forward solution
    print(f'Computing the forward solution ...')
    fwd = mne.make_forward_solution(raw_delta.info, trans=coreg.trans, src=src, bem=bem, meg=True,
                                    eeg=False, mindist=5.0, n_jobs=None, verbose=verbose)
    
    # ## computing the minimum-norm inverse solution
    print(f'Computing the minimum-norm inverse solution ...')
    inverse_operator = make_inverse_operator(raw_delta.info, fwd, noise_cov, loose=0.2, depth=0.8, verbose=verbose)

    ## compute source estimate object
    print(f'Computing the source estimate object ...')
    for raw_bp, title_raw in zip([raw_delta, raw_theta, raw_beta], ['delta', 'theta', 'beta']):

        stc = apply_inverse_raw(raw_bp, inverse_operator, lambda2, method=method, verbose=verbose)

        # ## morphing to template brain
        # morph = mne.compute_source_morph(stc, subject_from=f'0{subject}', subject_to="fsaverage",
        #                                 subjects_dir=subjects_dir, src_to=src_to)
        # stc_morph = morph.apply(stc)
    
        ## from source estimate object to brain parcels
        for atlas, title_atlas in zip(atlases, ['aparc', 'aparc.a2009s']):
            if atlas == 'aparc':
                labels = mne.read_labels_from_annot(subject=f'{subject}', parc=atlas,
                                                    subjects_dir=subjects_dir, verbose=verbose)
            if atlas == 'aparc.a2009s':
                labels = mne.read_labels_from_annot(subject=f'{subject}', parc=atlas,
                                                    subjects_dir=subjects_dir, verbose=verbose)[:-2]

            for mode in modes:
                tcs = stc.extract_label_time_course(labels, src, mode=mode, allow_empty=True, verbose=verbose)
                file_name = f'/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg3/subject_{subject}_{title_raw}_{title_atlas}_{mode}.npy'
                np.save(file=file_name, arr=tcs, allow_pickle=True)

    # # creating a report for each subjects
    # report = mne.Report(title=f'report_subject_{subject}', verbose=verbose)

    # report.add_raw(raw=raw, title='recording after preprocessing', butterfly=False, psd=False) 
    # report.add_figure(fig=ecg_evoked_meg, title='ECG evoked MEG', image_format='PNG')
    # report.add_figure(fig=ecg_evoked_grad, title='ECG evoked Gradiometer', image_format='PNG')
    # report.add_figure(fig=eog_evoked_meg, title='EOG evoked MEG', image_format='PNG')
    # report.add_figure(fig=eog_evoked_grad, title='EOG evoked Gradiometer', image_format='PNG')

    # if len(ecg_indices) > 0:
    #     report.add_figure(fig=ecg_component, title='ECG component', image_format='PNG')
    # if len(emg_indices) > 0:
    #     report.add_figure(fig=emg_component, title='EMG component', image_format='PNG')
    # if len(eog_indices) > 0:
    #     report.add_figure(fig=eog_component, title='EOG component (blink)', image_format='PNG')  
    # if len(saccade_indices) > 0:
    #     report.add_figure(fig=saccade_component, title='EOG component (saccade)', image_format='PNG') 

    # report.add_bem(subject=f'{subject}', subjects_dir=subjects_dir, title="MRI & BEM", decim=10, width=512)
    # report.add_trans(trans=fname_trans, info=raw.info, subject=f'{subject}',
    #                 subjects_dir=subjects_dir, alpha=1.0, title="Co-registration")
    
    # fname_report = f'/Users/payamsadeghishabestari/meg_gsp/reports/report_subject_{subject}.html'
    # report.save(fname=fname_report, open_browser=False, overwrite=True, verbose=verbose)
    # print(f'elapsed time for subject {subject} was {time.time() - start_time}')

In [ ]:
## only for subject 1045
subject = "1045"
raw_delta = mne.io.read_raw_fif(fname="/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg3/delta/1045_raw_tsss.fif", preload=True, verbose=False)
raw_theta = mne.io.read_raw_fif(fname="/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg3/theta/1045_raw_tsss.fif", preload=True, verbose=False)
raw_beta = mne.io.read_raw_fif(fname="/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg3/beta/1045_raw_tsss.fif", preload=True, verbose=False)

inverse_operator = mne.minimum_norm.read_inverse_operator(fname="/Users/payamsadeghishabestari/meg_gsp/stc_labels/1045-inv.fif", verbose=False)
src = mne.read_source_spaces(fname="/Users/payamsadeghishabestari/meg_gsp/stc_labels/1045-src.fif", verbose=False)

start, stop = raw_delta.time_as_index([100, 200])

for raw, raw_title in zip([raw_delta, raw_theta, raw_beta], ["delta", "theta", "beta"]): 
    stc = apply_inverse_raw(raw, inverse_operator, lambda2, method=method, start=start, stop=stop, verbose=True)
    
    for atlas, title_atlas in zip(atlases, ['aparc', 'aparc.a2009s']):
        if atlas == 'aparc':
            labels = mne.read_labels_from_annot(subject=f'{subject}', parc=atlas,
                                                subjects_dir=subjects_dir, verbose=True)
        if atlas == 'aparc.a2009s':
            labels = mne.read_labels_from_annot(subject=f'{subject}', parc=atlas,
                                                subjects_dir=subjects_dir, verbose=True)[:-2]
        for mode in modes:
            tcs = stc.extract_label_time_course(labels, src, mode=mode, allow_empty=True, verbose=verbose)
            file_name = f'/Users/payamsadeghishabestari/meg_gsp/stc_labels/subject_{subject}_{raw_title}_{title_atlas}_{mode}_2.npy'
            np.save(file=file_name, arr=tcs, allow_pickle=True)


for raw_title, title_atlas, mode in product(["delta", "theta", "beta"], ['aparc', 'aparc.a2009s'], ['mean', 'mean_flip']):
    arrs_list = []
    for i in range(1, 4):
        file_name = f'/Users/payamsadeghishabestari/meg_gsp/stc_labels/subject_{subject}_{raw_title}_{title_atlas}_{mode}_{i}.npy'
        arrs_list.append(np.load(file=file_name, allow_pickle=True))

    arr = np.concatenate(arrs_list, axis=1)
    np.save(f'/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg3/subject_{subject}_{raw_title}_{title_atlas}_{mode}.npy', arr)

Visualization of one single subject

In [ ]:
## learning graph
fname = '/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1/subject_539_alpha_aparc_mean.npy'
stc_in_labels = np.load(file=fname, allow_pickle=True)
graph_matrices = {}

graph_matrix = learn_graph.log_degree_barrier(X=stc_in_labels, dist_type='sqeuclidean', alpha=1,
                                            beta=1, step=0.5, w0=None, maxit=10000, rtol=1e-16,
                                            retall=False, verbosity='LOW')

graph_matrix = learn_graph.l2_degree_reg(X=stc_in_labels, dist_type='sqeuclidean', alpha=1,
                                        s=None, step=0.5, w0=None, maxit=10000, rtol=1e-16,
                                        retall=False, verbosity='LOW')

### extracting adjacency matrix
subject = 539
atlas = 'aparc'
subjects_dir = '/Applications/freesurfer/7.4.1/subjects'

if atlas == 'aparc':
    labels = mne.read_labels_from_annot(subject="fsaverage", parc=atlas,
                                        subjects_dir=subjects_dir, verbose=False)[:-1]
if atlas == 'aparc.a2009s':
    labels = mne.read_labels_from_annot(subject="fsaverage", parc=atlas,
                                        subjects_dir=subjects_dir, verbose=False)[:-2]
node_coords = []
for label in labels:
    if label.hemi == 'lh':
        hemi = 0
    if label.hemi == 'rh':
        hemi = 1
    center_vertex = label.center_of_mass(subject="fsaverage", 
                                        restrict_vertices=False, 
                                        subjects_dir=None)
    mni_pos = mne.vertex_to_mni(center_vertex, hemis=hemi,
                            subject="fsaverage", subjects_dir=None)
    node_coords.append(mni_pos)

node_coords = np.array(node_coords)
ticks = [lb.name for lb in labels]

### visualizing
corr_matrix = np.corrcoef(graph_matrix, rowvar=False)
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
sns.heatmap(graph_matrix, cmap="hot", annot=False, xticklabels=ticks, yticklabels=ticks)

fig, axes = plt.subplots(1, 1, figsize=(11, 5))
plot_connectome(adjacency_matrix=graph_matrix, node_coords=node_coords,
                node_color='k', node_size=20, axes=axes,
                edge_threshold='90%')
fig.tight_layout()

In [ ]:
## plot it for all subjects
subjects_dir = '/Applications/freesurfer/7.4.1/subjects'
dir_control = '/Users/payamsadeghishabestari/KI_MEG/KI_RS/tinmeg1' 
dir_case = '/Users/payamsadeghishabestari/KI_MEG/KI_RS/tinmeg3' 
subjects_control = sorted(os.listdir(dir_control))[1:]
subjects_case = sorted(os.listdir(dir_case))[1:]
subjects = subjects_control + subjects_case
freqs = ["alpha", "gamma"]
atlases = ['aparc', 'aparc.a2009s']

for (subject, freq, atlas) in tqdm(product(subjects, freqs, atlases)):
    
    if subject in subjects_control:
        fname = f'/Users/payamsadeghishabestari/meg_gsp/stc_labels_morphed/tinmeg1/subject_{subject}_{freq}_{atlas}_mean.npy'
    if subject in subjects_case:
        fname = f'/Users/payamsadeghishabestari/meg_gsp/stc_labels_morphed/tinmeg3/subject_{subject}_{freq}_{atlas}_mean.npy'
    
    if atlas == 'aparc':
        stc_in_labels = np.load(file=fname, allow_pickle=True)[:-1,:] # mind this

    if atlas == 'aparc.a2009s':
        stc_in_labels = np.load(file=fname, allow_pickle=True)
    
    graph_matrices = {}
    graph_matrix = learn_graph.log_degree_barrier(X=stc_in_labels, dist_type='sqeuclidean', alpha=1,
                                                beta=1, step=0.5, w0=None, maxit=10000, rtol=1e-16,
                                                retall=False, verbosity='LOW')
    ### extracting adjacency matrix
    if subject in subjects_control:
        subject = f"0{subject}"
        save_dir = f"/Users/payamsadeghishabestari/meg_gsp/all_subjects_graph_morphed/{atlas}/{freq}/control/{subject}.png"
    else:
        save_dir = f"/Users/payamsadeghishabestari/meg_gsp/all_subjects_graph_morphed/{atlas}/{freq}/case/{subject}.png"

    if atlas == 'aparc':
        labels = mne.read_labels_from_annot(subject="fsaverage", parc=atlas,
                                            subjects_dir=subjects_dir, verbose=False)[:-1]
    if atlas == 'aparc.a2009s':
        labels = mne.read_labels_from_annot(subject="fsaverage", parc=atlas,
                                            subjects_dir=subjects_dir, verbose=False)[:-2]
    
    
    node_coords = []
    for label in labels:
        if label.hemi == 'lh':
            hemi = 0
        if label.hemi == 'rh':
            hemi = 1
        center_vertex = label.center_of_mass(subject="fsaverage",
                                            restrict_vertices=False, 
                                            subjects_dir=subjects_dir)
        mni_pos = mne.vertex_to_mni(center_vertex, hemis=hemi,
                                subject="fsaverage", subjects_dir=subjects_dir)
        node_coords.append(mni_pos)

    node_coords = np.array(node_coords)
    ticks = [lb.name for lb in labels]

    ### visualizing
    fig, axes = plt.subplots(1, 1, figsize=(11, 5))
    plot_connectome(adjacency_matrix=graph_matrix, node_coords=node_coords,
                    node_color='k', node_size=20, axes=axes,
                    edge_threshold='90%')
    fig.tight_layout()
    fig.savefig(save_dir, format="png")

Creating Identifiability Matrix

In [ ]:
def compute_identifiability_matrix(directory, atlas, method, freq_range, end_sample, graph_based=True,
                                    con_method = "pli", graph_method="log", case_or_control="both",
                                    dist_type="sqeuclidean", alpha=1, beta=1, step=0.5, max_iter=10000,
                                    rtol=1e-16, subtract_off_diag_avg=True, verbose=None):
    
    ## reorder labels
    if atlas == 'aparc':
        n_labels = 68
    if atlas == 'aparc.a2009s':
        n_labels = 148 

    ## loading all subjects
    fnames = []
    if case_or_control == 'control':
        for filename in sorted(os.listdir(directory)): 
            f = os.path.join(directory, filename)
            if os.path.isfile(f) and f.endswith(f'{atlas}_{method}.npy') and freq_range in f:
                fnames.append(f)

    if case_or_control == 'case':
        for filename in sorted(os.listdir(directory)): 
            f = os.path.join(directory, filename)
            if os.path.isfile(f) and f.endswith(f'{atlas}_{method}.npy') and freq_range in f:
                fnames.append(f)

    if case_or_control == 'both':
        for dir in directory:
            for filename in sorted(os.listdir(dir)): 
                f = os.path.join(dir, filename)
                if os.path.isfile(f) and f.endswith(f'{atlas}_{method}.npy') and freq_range in f:
                    fnames.append(f)

    ## reading the files and reordering labels
    n_subjects = len(fnames)
    half_size = end_sample // 2
    all_tcs_s1 = []
    all_tcs_s2 = []
    for fname in fnames:
        tcs = np.load(file=fname, allow_pickle=True)
        all_tcs_s1.append(tcs[:, :half_size])  # session 1
        all_tcs_s2.append(tcs[:, half_size:end_sample]) # session 2

    vectors_s1 = []
    vectors_s2 = []
    up_tri_idxs = np.triu_indices(n_labels, k=1)

    for tc1, tc2 in tzip(all_tcs_s1, all_tcs_s2):
        
        if graph_based: # learning the graph matrix for both sessions
            if graph_method == "log":
                graph_s1 = learn_graph.log_degree_barrier(X=tc1, dist_type=dist_type,
                                                        alpha=alpha, beta=beta, step=step,
                                                        w0=None, maxit=max_iter, rtol=rtol,
                                                        retall=False, verbosity=verbose)
                graph_s2 = learn_graph.log_degree_barrier(X=tc2, dist_type=dist_type,
                                                        alpha=alpha, beta=beta, step=step,
                                                        w0=None, maxit=max_iter, rtol=rtol,
                                                        retall=False, verbosity=verbose)
                
            if graph_method == "glasso":
                graph_s1 = learn_graph.glasso(X=tc1, alpha=alpha, w0=None, maxit=max_iter,
                                            rtol=rtol, retall=False, verbosity=verbose)
                graph_s2 = learn_graph.glasso(X=tc2, alpha=alpha, w0=None, maxit=max_iter,
                                            rtol=rtol, retall=False, verbosity=verbose)
                
            if graph_method == "l2-reg":
                graph_s1 = learn_graph.l2_degree_reg(X=tc1, dist_type=dist_type, alpha=alpha,
                                                    s=None, step=step, w0=None, maxit=max_iter,
                                                    rtol=rtol, retall=False, verbosity=verbose)
                graph_s2 = learn_graph.l2_degree_reg(X=tc2, dist_type=dist_type, alpha=alpha,
                                                    s=None, step=step, w0=None, maxit=max_iter,
                                                    rtol=rtol, retall=False, verbosity=verbose)
                
        if not graph_based:   
            if freq_range == "alpha":
                (l_freq, h_freq) = (8, 13)
            if freq_range == "gamma":
                (l_freq, h_freq) = (30, 45)
        
            freqs = np.linspace(l_freq, h_freq, 5)
            con_s1 = spectral_connectivity_time(data=tc1[np.newaxis, :], method=con_method, freqs=freqs,
                                                average=True, mode="multitaper", sfreq=sfreq, fmin=l_freq,
                                                fmax=h_freq, faverage=True, verbose=False)
            con_s2 = spectral_connectivity_time(data=tc2[np.newaxis, :], method=con_method, freqs=freqs,
                                                average=True, mode="multitaper", sfreq=sfreq, fmin=l_freq, 
                                                fmax=h_freq, faverage=True, verbose=False)
            
            graph_s1 = con_s1.get_data(output="dense")[:, :, 0] + con_s1.get_data(output="dense")[:, :, 0].T
            graph_s2 = con_s2.get_data(output="dense")[:, :, 0] + con_s2.get_data(output="dense")[:, :, 0].T
        
        ## vectorizing the upper triangle and computing the correlation coeff
        vectors_s1.append(graph_s1[up_tri_idxs])
        vectors_s2.append(graph_s2[up_tri_idxs])
        
    ## filling identifiability matrix
    id_matrix = np.zeros(shape=(n_subjects, n_subjects))
    for i, j in itertools.product(range(n_subjects), range(n_subjects)):
        id_matrix[i][j] = np.corrcoef(vectors_s1[i], vectors_s2[j])[0, 1]

    if subtract_off_diag_avg:
        for i, j in itertools.product(range(n_subjects), range(n_subjects)):
            if i == j:
                id_matrix[i][j] = id_matrix[i][j] - (sum(id_matrix[i]) - id_matrix[i][j]) / (n_subjects - 1)

    return id_matrix

def compute_rank(id_matrix):
    ranks = []
    for row_idx, row in enumerate(id_matrix):
        ranks.append(np.where(sorted(row, reverse=True) == row[row_idx])[0][0] + 1)   
    ranks_control = ranks[:23]
    ranks_case = ranks[23:]
    return ranks_control, ranks_case 

Example Run for one condition

In [ ]:
dir1 = '/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1'
dir2 = '/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg3'
method = 'mean' # or mean_flip
case_or_control = 'both' # or control or both
sfreq = 250
end_sample = 5 * 60 * sfreq 
dist_type = "sqeuclidean"
alpha = 1
step = 0.5
max_iter = 10000
rtol = 1e-16
beta = 1
atlas = 'aparc'
freq_range = 'alpha'
id_matrix = compute_identifiability_matrix(directory=[dir1, dir2], atlas=atlas, method=method, graph_based=False,
                                        con_method = "pli", freq_range=freq_range, end_sample=end_sample,
                                        graph_method="log", case_or_control="both", dist_type=dist_type, alpha=alpha,
                                        beta=beta, step=step, max_iter=max_iter, rtol=rtol, 
                                        subtract_off_diag_avg=False, verbose="NONE")
plt.figure()
plt.imshow(id_matrix)
plt.colorbar()

Making big Dictionary

In [ ]:
dir1 = '/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1'
dir2 = '/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg3'
method = 'mean' # or mean_flip
case_or_control = 'both' # or control or both
sfreq = 250
end_sample = 5 * 60 * sfreq 
dist_type = "sqeuclidean"
alpha = 1
step = 0.5
max_iter = 10000
rtol = 1e-16
betas = [0.5, 1, 1.5]
atlases = ['aparc', 'aparc.a2009s']
freq_ranges = ['alpha', 'gamma']

identifiability_dict = {}
for freq_range in freq_ranges:
    identifiability_dict[freq_range] = {}
    for atlas in atlases:
        identifiability_dict[freq_range][atlas] = {}
        for beta in betas:
            id_matrix = compute_identifiability_matrix(directory=[dir1, dir2], atlas=atlas, method=method,
                                        freq_range=freq_range, end_sample=end_sample, graph_method="log",
                                        case_or_control="both", dist_type=dist_type, alpha=alpha,
                                        subtract_off_diag_avg=False, beta=beta, step=step, max_iter=max_iter,
                                        rtol=rtol, verbose="NONE")
            identifiability_dict[freq_range][atlas][beta] = id_matrix

with open('/Users/payamsadeghishabestari/meg_gsp/identifiability_no_subtract.pkl', 'wb') as file:
    pickle.dump(identifiability_dict, file)

Visualizing it

In [ ]:
## loading
file_path = '/Users/payamsadeghishabestari/meg_gsp/identifiability_no_subtract.pkl'
with open(file_path, 'rb') as file:
    identifiability_dict = pickle.load(file)

line_x = np.linspace(-0.5, 41.5, 100)

## visualizing 
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
ax.plot(line_x, [17.5]*len(line_x), 'grey', linewidth=3)
ax.plot([17.5]*len(line_x), line_x, 'grey', linewidth=3)

ax.plot(line_x, [40.5]*len(line_x), 'grey', linewidth=3)
ax.plot([40.5]*len(line_x), line_x, 'grey', linewidth=3)
ax.plot(line_x, [-0.5]*len(line_x), 'grey', linewidth=3)
ax.plot([-0.5]*len(line_x), line_x, 'grey', linewidth=3)

im = ax.imshow(identifiability_dict['gamma']['aparc'][1], cmap="coolwarm", vmin=0, vmax=1)
divider = make_axes_locatable(ax)
# cax = divider.append_axes('right', size='5%', pad=0.1)
# fig.colorbar(im, cax=cax, orientation='vertical')
ax.set_xticks([])
ax.set_yticks([])
ax.axis('off')
# fig.savefig(fname="/Users/payamsadeghishabestari/meg_gsp/papar_plots/id_matrix_alpha.svg")

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(10, 6))

id_matrix = identifiability_dict['gamma']['aparc'][0.5]
ranks_control, ranks_case = compute_rank(id_matrix)
ranks_co = [ranks_control.count(i) for i in range(1, 42)]
ranks_ca = [ranks_case.count(i) for i in range(1, 42)]
bar1 = axs[0][0].bar(range(1, 42), ranks_co, label='Control')
bar2 = axs[0][0].bar(range(1, 42), ranks_ca, label='Case', bottom=ranks_co)

id_matrix = identifiability_dict['gamma']['aparc'][1]
ranks_control, ranks_case = compute_rank(id_matrix)
ranks_co = [ranks_control.count(i) for i in range(1, 42)]
ranks_ca = [ranks_case.count(i) for i in range(1, 42)]
bar1 = axs[1][0].bar(range(1, 42), ranks_co, label='Control')
bar2 = axs[1][0].bar(range(1, 42), ranks_ca, label='Case', bottom=ranks_co)

id_matrix = identifiability_dict['gamma']['aparc'][1.5]
ranks_control, ranks_case = compute_rank(id_matrix)
ranks_co = [ranks_control.count(i) for i in range(1, 42)]
ranks_ca = [ranks_case.count(i) for i in range(1, 42)]
bar1 = axs[2][0].bar(range(1, 42), ranks_co, label='Control')
bar2 = axs[2][0].bar(range(1, 42), ranks_ca, label='Case', bottom=ranks_co)

id_matrix = identifiability_dict['alpha']['aparc.a2009s'][0.5]
ranks_control, ranks_case = compute_rank(id_matrix)
ranks_co = [ranks_control.count(i) for i in range(1, 42)]
ranks_ca = [ranks_case.count(i) for i in range(1, 42)]
bar1 = axs[0][1].bar(range(1, 42), ranks_co, label='Control')
bar2 = axs[0][1].bar(range(1, 42), ranks_ca, label='Case', bottom=ranks_co)

id_matrix = identifiability_dict['alpha']['aparc.a2009s'][1]
ranks_control, ranks_case = compute_rank(id_matrix)
ranks_co = [ranks_control.count(i) for i in range(1, 42)]
ranks_ca = [ranks_case.count(i) for i in range(1, 42)]
bar1 = axs[1][1].bar(range(1, 42), ranks_co, label='Control')
bar2 = axs[1][1].bar(range(1, 42), ranks_ca, label='Case', bottom=ranks_co)

id_matrix = identifiability_dict['alpha']['aparc.a2009s'][1.5]
ranks_control, ranks_case = compute_rank(id_matrix)
ranks_co = [ranks_control.count(i) for i in range(1, 42)]
ranks_ca = [ranks_case.count(i) for i in range(1, 42)]
bar1 = axs[2][1].bar(range(1, 42), ranks_co, label='Control')
bar2 = axs[2][1].bar(range(1, 42), ranks_ca, label='Case', bottom=ranks_co)

for i, j in itertools.product(range(3), range(2)):
    axs[i][j].set_xticks(range(1, 10))
    axs[i][j].set_xlim([0, 10])
    axs[i][j].set_ylim([0, 41])
    axs[i][j].legend(loc='upper right', frameon=False)

axs[0][0].set_title('aparc (68 lbs)')
axs[0][1].set_title('aparc (148 lbs)')
axs[0][0].set_ylabel('beta = 0.5')
axs[1][0].set_ylabel('beta = 1.0')
axs[2][0].set_ylabel('beta = 1.5')
axs[2][0].set_xlabel('ranks')
axs[2][1].set_xlabel('ranks')


Visualizing ranks

In [ ]:
colors = sns.color_palette('Set1')[:2]
X = ['Rank 1','Rank 2','Rank 3','Rank 4', "Rank 5"] 
X_axis = np.arange(len(X)) 

## alpha (0.5)
controls_rank = [23,0,0,0,0] 
cases_rank = [16,0,0,2,0] 
fig, ax = plt.subplots(1, 1, figsize=(6, 3))
ax.bar(X_axis - 0.2, cases_rank, 0.4, label = 'Girls', color=colors[0]) 
ax.bar(X_axis + 0.2, controls_rank, 0.4, label = 'Boys', color=colors[1]) 
ax.set_xticks(X_axis, X) 
ax.set_title("freq_alpha, beta=0.5") 
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_ylim([0, 25])

## alpha (1)
controls_rank = [22,1,0,0,0] 
cases_rank = [16,1,1,0,0] 
fig, ax = plt.subplots(1, 1, figsize=(6, 3))
ax.bar(X_axis - 0.2, cases_rank, 0.4, label = 'Girls', color=colors[0]) 
ax.bar(X_axis + 0.2, controls_rank, 0.4, label = 'Boys', color=colors[1]) 
ax.set_xticks(X_axis, X) 
ax.set_title("freq_alpha, beta=1") 
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_ylim([0, 25])

## alpha (1.5)
controls_rank = [22,1,0,0,0] 
cases_rank = [16,1,1,0,0] 
fig, ax = plt.subplots(1, 1, figsize=(6, 3))
ax.bar(X_axis - 0.2, cases_rank, 0.4, label = 'Case', color=colors[0]) 
ax.bar(X_axis + 0.2, controls_rank, 0.4, label = 'Control', color=colors[1]) 
ax.set_xticks(X_axis, X) 
ax.set_title("freq_alpha, beta=1.5") 
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(frameon=False)
ax.set_ylim([0, 25])

In [ ]:
fname = "/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1/subject_756_alpha_aparc_mean.npy"
tcs = np.load(file=fname, allow_pickle=True)
half_size = int(len(tcs[0]) / 2)
tc1 = tcs[:, :half_size]  # session 1
tc2 = tcs[:, half_size:] # session 2

dist_type = "sqeuclidean"
alpha = 1
beta = 1
step = 0.5
max_iter = 10000
rtol = 1e-16

graph_s1 = learn_graph.log_degree_barrier(X=tc1, dist_type=dist_type,
                                        alpha=alpha, beta=beta, step=step,
                                        w0=None, maxit=max_iter, rtol=rtol,
                                        retall=False)
graph_s2 = learn_graph.log_degree_barrier(X=tc2, dist_type=dist_type,
                                        alpha=alpha, beta=beta, step=step,
                                        w0=None, maxit=max_iter, rtol=rtol,
                                        retall=False)

In [ ]:
A = graph_s1 / graph_s1.max()
mask1 = np.tri(A.shape[0], k=-1)
A = np.ma.array(A, mask=1-mask1) # mask out the lower triangle

B = graph_s2 / graph_s2.max()
mask2 = np.tri(B.shape[0], k=0)
B = np.ma.array(B, mask=mask2) # mask out the lower triangle

fig, ax1 = plt.subplots(1,1)

im1 = ax1.imshow(A, cmap="afmhot", vmin=0, vmax=0.7)
im1 = ax1.imshow(B, cmap="afmhot", vmin=0, vmax=0.7)

ax1.spines['left'].set_visible(False)
ax1.spines['bottom'].set_visible(False)
ax1.axis('off')
fig.colorbar(mappable=im1, ax=ax1)
plt.show()


Visualising brains

In [ ]:
# extra plot
Brain = mne.viz.get_brain_class()
clr = 0.85
brain_kwargs = dict(alpha=1, background="white", cortex=[(clr, clr, clr), (clr, clr, clr)], size=(800, 600), views='lateral')
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 69 labels
brain = Brain("fsaverage", hemi="lh", surf="pial_semi_inflated", **brain_kwargs)
brain.add_annotation(annot="aparc", borders=False, alpha=1)

Utilizing UMAP

In [ ]:
# loading subjects
atlas = 'aparc'
freq_range = 'alpha'
method = 'mean'
dirs = ['/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1',
        '/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg3']
fnames = []
for dir in dirs:
    for filename in sorted(os.listdir(dir)): 
        f = os.path.join(dir, filename)
        if os.path.isfile(f) and f.endswith(f'{atlas}_{method}.npy') and freq_range in f:
            fnames.append(f)

# creating vectors for each time period
print(f'Computing the Graph Matrixes ...')
sfreq = 250
init_time = 2
end_time = 300
n_frames = 100
if atlas == 'aparc':
    n_labels = 68
if atlas == 'aparc.a2009s':
    n_labels = 148
up_tri_idxs = np.triu_indices(n_labels, k=1)
durations = (np.linspace(init_time, end_time, n_frames) * sfreq).astype(int)
all_vectors = []
for fname in tqdm(fnames):
    
    # vectors = np.zeros(shape=(len(durations), len(up_tri_idxs[0])))
    vectors = np.zeros(shape=(len(durations), len(label_idxs))) # run this only if you want to use significant edges
    
    stc_in_labels = np.load(file=fname, allow_pickle=True)
    for idx, duration in enumerate(durations):
        graph = learn_graph.log_degree_barrier(X=stc_in_labels[:, :duration],
                                                dist_type='sqeuclidean', alpha=1,
                                                beta=1, step=0.5, w0=None, maxit=10000,
                                                rtol=1e-16, retall=False, verbosity='NONE')
        graph = graph / np.linalg.norm(graph, 'fro') # normalizing it
        
        # vectors[idx] = graph[up_tri_idxs]
        vectors[idx] = graph[tuple(np.array(label_idxs).T)] # run this only if you want to use significant edges
    
    all_vectors.append(vectors)

all_vectors = np.array(all_vectors)

# going to umap space
print(f'Creating the UMAP space data ...')
data_transformed = []
for i in tqdm(range(n_frames)):
    data = torch.tensor(all_vectors[:,i,:], dtype=torch.float32)
    reshaped_data = data.reshape(data.shape[0], -1)
    umap_transform = UMAP(n_components=2, n_neighbors=3, 
                    learning_rate = 1e-7, metric='euclidean')
    data_transformed.append(umap_transform.fit_transform(reshaped_data))

# creating the animation
colors = ['b'] * 23 + ['r'] * 18 # control vs case
fig, ax = plt.subplots()
ax.set_xlim(-2, 12)
ax.set_ylim(-2, 12)
ax.grid()
ax.set_xlabel('UMAP componet 1')
ax.set_ylabel('UMAP componet 2')
scatter = ax.scatter([], [], s=25)
text = ax.text(-1.5, -1.5, '', fontsize='large', fontstyle='italic')
def update(frame):
    x = data_transformed[frame][:, 0]
    y = data_transformed[frame][:, 1]
    offsets = np.column_stack((x, y))
    scatter.set_offsets(offsets)
    scatter.set_color(colors)
    text.set_text(f'Frame {frame}/{n_frames}')
    return scatter,

animation = FuncAnimation(fig, update, frames=n_frames, init_func=None, repeat=False, interval=300)
plt.show()

# saving the animation
animation.save(f"/Users/payamsadeghishabestari/meg_gsp/{freq_range}_{atlas}_{method}_significant_umap.gif", writer="pillow")
plt.close(animation._fig)


Utilizing 2 component PCA

In [ ]:
# loading subjects
atlas = 'aparc'
freq_range = 'alpha'
method = 'mean'
dirs = ['/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1',
        '/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg3']
fnames = []
for dir in dirs:
    for filename in sorted(os.listdir(dir)): 
        f = os.path.join(dir, filename)
        if os.path.isfile(f) and f.endswith(f'{atlas}_{method}.npy') and freq_range in f:
            fnames.append(f)

# creating vectors for each time period
print(f'Computing the Graph Matrixes ...')
sfreq = 250
init_time = 2
end_time = 300
n_frames = 6
if atlas == 'aparc':
    n_labels = 68
if atlas == 'aparc.a2009s':
    n_labels = 148
up_tri_idxs = np.triu_indices(n_labels, k=1)
durations = (np.linspace(init_time, end_time, n_frames) * sfreq).astype(int)
all_vectors = []
for fname in tqdm(fnames):
    vectors = np.zeros(shape=(len(durations), len(up_tri_idxs[0])))
    # vectors = np.zeros(shape=(len(durations), len(label_idxs))) # run this only if you want to use significant edges
    
    stc_in_labels = np.load(file=fname, allow_pickle=True)
    for idx, duration in enumerate(durations):
        graph = learn_graph.log_degree_barrier(X=stc_in_labels[:, :duration],
                                                dist_type='sqeuclidean', alpha=1,
                                                beta=1, step=0.5, w0=None, maxit=10000,
                                                rtol=1e-16, retall=False, verbosity='NONE')
        graph = graph / np.linalg.norm(graph, 'fro') # normalizing it
        
        vectors[idx] = graph[up_tri_idxs]
        # vectors[idx] = graph[tuple(np.array(label_idxs).T)] # run this only if you want to use significant edges

    all_vectors.append(vectors)

all_vectors = np.array(all_vectors)

# computing 2 - PCA 
print(f'Computing the PCA ...')
data_transformed = []
for i in tqdm(range(n_frames)):
    pca = PCA(n_components=2)
    data_transformed.append(pca.fit_transform(all_vectors[:,i,:]))

# creating the animation
colors = ['b'] * 23 + ['r'] * 18 # control vs case
fig, ax = plt.subplots()
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.grid()
ax.set_xlabel('PCA componet 1')
ax.set_ylabel('PCA componet 2')
scatter = ax.scatter([], [], s=25)
text = ax.text(-4.5, -4.5, '', fontsize='large', fontstyle='italic')
def update(frame):
    x = data_transformed[frame][:, 0]
    y = data_transformed[frame][:, 1]
    offsets = np.column_stack((x, y))
    scatter.set_offsets(offsets)
    scatter.set_color(colors)
    text.set_text(f'Frame {frame}/{n_frames}')
    return scatter,

animation = FuncAnimation(fig, update, frames=n_frames, init_func=None, repeat=False, interval=300)
plt.show()


In [ ]:
# creating the animation
colors = ['b'] * 23 + ['r'] * 18 # control vs case
fig, ax = plt.subplots()
ax.set_xlim(-.5, .5)
ax.set_ylim(-.5, .5)
ax.grid()
ax.set_xlabel('PCA componet 1')
ax.set_ylabel('PCA componet 2')
scatter = ax.scatter([], [], s=25)
text = ax.text(-0.5, -0.5, '', fontsize='large', fontstyle='italic')
def update(frame):
    x = data_transformed[frame][:, 0]
    y = data_transformed[frame][:, 1]
    offsets = np.column_stack((x, y))
    scatter.set_offsets(offsets)
    scatter.set_color(colors)
    text.set_text(f'Frame {frame}/{n_frames}')
    return scatter,

animation = FuncAnimation(fig, update, frames=n_frames, init_func=None, repeat=False, interval=300)
plt.show()

Compare networks statistically between two groups (t-test)

In [ ]:
## parameters to adjust
alpha_param = 1
beta_param = 1
dist_type = "sqeuclidean"

statistics = "t-test" # "Mann-Whitney U test"
F = 300
sfreq = 250
alpha = 0.05  # significance level
p_thr = 0.05
end_sample = F * sfreq

# loading subjects
atlas = 'aparc'
freq_ranges = ["delta", "theta", "alpha", "beta", "gamma"]
method = 'mean'
dirs = ['/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1',
        '/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg3']

labels = mne.read_labels_from_annot(subject='fsaverage', parc=atlas, subjects_dir=None, verbose=False)[:-1]
lb_names = np.array([lb.name for lb in labels])

stats_dict = {}
pvals_dict = {}
for freq_range in freq_ranges[4:]:    
    fnames = []
    for dir in dirs:
        for filename in sorted(os.listdir(dir)): 
            f = os.path.join(dir, filename)
            if os.path.isfile(f) and f.endswith(f'{atlas}_{method}.npy') and freq_range in f:
                fnames.append(f)

    # creating graphs and vectors
    print(f'Computing the Graph Matrixes ...')
    if atlas == 'aparc':
        n_labels = 68
    if atlas == 'aparc.a2009s':
        n_labels = 148
    up_tri_idxs = np.triu_indices(n_labels, k=1)
    vectors = []
    for fname in tqdm(fnames):
        stc_in_labels = np.load(file=fname, allow_pickle=True)
        graph = learn_graph.log_degree_barrier(X=stc_in_labels[:,:end_sample], dist_type=dist_type,
                                            alpha=alpha_param, beta=beta_param, step=0.5, w0=None, maxit=10000,
                                            rtol=1e-16, retall=False, verbosity='NONE')
        graph = graph / np.linalg.norm(graph, 'fro') # normalizing it
        vector = graph[up_tri_idxs]
        vectors.append(vector)
    vectors = np.array(vectors)

    # removing very small connections
    for i in range(len(fnames)):
        zero_edges = np.where(vectors[i] < vectors[i].max() * 0.01)[0]
        vectors[i][zero_edges] = 0

    # statistics
    control_group = vectors[:23,:]
    case_group = vectors[23:,:]
    if statistics == 't-test':
        stat, p_values = ttest_ind(control_group, case_group,
                                    permutations=0, random_state=42)
    if statistics == 'Mann-Whitney U test':
        stat, p_values = mannwhitneyu(control_group, case_group)

    my_stat_dict = {}
    p_dict = {}
    methods = ['bonferroni', 'fdr_bh']
    for method in methods:
        reject_null, p_corrected, _, _ = sm.stats.multipletests(pvals=p_values, alpha=alpha,
                                                            method=method)
        label_idxs = []
        if len(np.where(p_corrected < p_thr)[0]) == 0:
            print(f"No statistical difference between brain labels for method {method}")
        else:
            for idx in np.where(p_corrected < p_thr)[0]:
                label_idxs.append((up_tri_idxs[0][idx], up_tri_idxs[1][idx]))
        my_stat_dict[method] = lb_names[label_idxs]
        p_dict[method] = p_corrected
    
    stats_dict[freq_range] = my_stat_dict
    pvals_dict[freq_range] = p_dict

In [ ]:
print(stats_dict["gamma"]["fdr_bh"])

Check the distribution of significant edges

In [ ]:
my_dict = {"group": [], "connection": []}
up_tri_idxs = np.triu_indices(n_labels, k=1)
(lb1, lb2) = ("insula-lh", "parahippocampal-lh")

lb1_idx = list(lb_names).index(lb1)
lb2_idx = list(lb_names).index(lb2)
idx = np.where((up_tri_idxs[0] == min(lb1_idx, lb2_idx)) & (up_tri_idxs[1] == max(lb1_idx, lb2_idx)))[0]

for sub in control_group:
    my_dict["connection"].append(sub[idx][0])
    my_dict["group"].append("control")
for sub in case_group:
    my_dict["connection"].append(sub[idx][0])
    my_dict["group"].append("case")

df = pd.DataFrame(my_dict)

palette_color = ['#1f77b4', '#d62728']
fig, ax = plt.subplots(1, 1, figsize=(6, 3))
sns.violinplot(data=df, x="group", y="connection", saturation=0.75, palette=palette_color, fill=False, ax=ax, inner=None)
sns.swarmplot(data=df, x="group", y="connection", palette=palette_color, ax=ax)
ax.set_title(f"connection between {lb1} vs {lb2}")
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)

Compare networks statistically between two groups (k-sample)

In [ ]:
## parameters to adjust
alpha_param = 1
beta_param = 1
dist_type = "sqeuclidean"

statistics = "t-test" # "Mann-Whitney U test"
F = 300
sfreq = 250
alpha = 0.05  # significance level
p_thr = 0.05
end_sample = F * sfreq

# loading subjects
atlas = 'aparc'
freq_ranges = ["delta", "theta", "alpha", "beta", "gamma"]
method = 'mean'
dirs = ['/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1',
        '/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg3']

labels = mne.read_labels_from_annot(subject='fsaverage', parc=atlas, subjects_dir=None, verbose=False)[:-1]
lb_names = np.array([lb.name for lb in labels])

stats_dict = {}
pvals_dict = {}
for freq_range in freq_ranges[2:3]:    
    fnames = []
    for dir in dirs:
        for filename in sorted(os.listdir(dir)): 
            f = os.path.join(dir, filename)
            if os.path.isfile(f) and f.endswith(f'{atlas}_{method}.npy') and freq_range in f:
                fnames.append(f)

    # creating graphs and vectors
    print(f'Computing the Graph Matrixes ...')
    if atlas == 'aparc':
        n_labels = 68
    if atlas == 'aparc.a2009s':
        n_labels = 148
    up_tri_idxs = np.triu_indices(n_labels, k=1)
    graphs = []
    for fname in tqdm(fnames):
        stc_in_labels = np.load(file=fname, allow_pickle=True)
        graph = learn_graph.log_degree_barrier(X=stc_in_labels[:,:end_sample], dist_type=dist_type,
                                            alpha=alpha_param, beta=beta_param, step=0.5, w0=None, maxit=10000,
                                            rtol=1e-16, retall=False, verbosity='NONE')
        graph = graph / np.linalg.norm(graph, 'fro') # normalizing it
        
        thr = 0.01 * np.max(graph)
        graph[graph < thr] = 0

        graphs.append(graph)
    graphs = np.array(graphs)

In [ ]:
method = "bonferroni"
samples_co = graphs[:23]
samples_ca = graphs[23:]
connectomes = [samples_co, samples_ca]

indices = zip(*np.triu_indices(68, 1))
edge_pvals = []
for roi_i, roi_j in indices:

    # Get the (i,j)-th edge for each connectome
    samples = [type[:, roi_i, roi_j] for type in connectomes]

    # Calculate the p-value for the (i,j)-th edge
    try:
        statistic, pvalue = KSample("Dcorr").test(*samples, reps=10000, workers=-1)
    except ValueError:
        # A ValueError is thrown when any of the samples have equal edge
        # weights (i.e. one of the inputs has 0 variance)
        statistic = np.nan
        pvalue = 1

    edge_pvals.append([roi_i, roi_j, statistic, pvalue])

# Convert the nested list to a dataframe
edge_pvals_df = pd.DataFrame(edge_pvals, columns=["ROI_1", "ROI_2", "stat", "pval"])

# multiple comparison
alpha = 0.05
ps = [item[3] for item in edge_pvals]
reject_null, corr_ps, _, _ = sm.stats.multipletests(pvals=ps, alpha=alpha,
                                                        method=method)
for p_idx, p in enumerate(corr_ps):
    edge_pvals[p_idx][3] = p

edge_pvals_df["corr_pval"] = corr_ps
edge_pvals_df["significant"] = (edge_pvals_df["corr_pval"] < alpha)

# Get the top 10 strongest signal edges
edge_pvals_df.sort_values(by="corr_pval", inplace=True, ignore_index=True)
edge_pvals_top = edge_pvals_df.head(10)

# Replace ROI indices with actual names
labels = mne.read_labels_from_annot(subject="fsaverage", parc="aparc",
                                    subjects_dir=None, verbose=False)[:-1]
ticks = [lb.name for lb in labels]
def lookup_roi_name(index):
    roi_name = ticks[index]
    return f"{roi_name}"

edge_pvals_top["ROI_1"] = edge_pvals_top["ROI_1"].apply(lookup_roi_name)
edge_pvals_top["ROI_2"] = edge_pvals_top["ROI_2"].apply(lookup_roi_name)
edge_pvals_top.head()

In [ ]:
## GMM check
two_groups = np.concatenate((control_group, case_group), axis=0)
aics_1 = []; aics_2 = []
bics_1 = []; bics_2 = []
for edge in tqdm(two_groups.T):
    edges = edge.reshape(-1, 1)
    # 1 component
    gmm = GaussianMixture(n_components=1)
    gmm.fit(edges)
    aics_1.append(gmm.aic(edges))
    bics_1.append(gmm.bic(edges))
    # 2 component
    gmm = GaussianMixture(n_components=2)
    gmm.fit(edges)
    aics_2.append(gmm.aic(edges))
    bics_2.append(gmm.bic(edges))

label_idxs = []
for idx, (aic_1, aic_2) in enumerate(zip(aics_1, aics_2)):
    if aic_2 < (aic_1 * 1.2): # can put more constraint here now 10 %
        label_idxs.append((up_tri_idxs[0][idx], up_tri_idxs[1][idx]))

In [ ]:
# visualization
control_avg_vector = np.mean(vectors[:23,:], axis=0)
control_avg_graph = np.zeros(shape=(n_labels, n_labels))
control_avg_graph[up_tri_idxs] = control_avg_vector
control_avg_graph = control_avg_graph + control_avg_graph.T
control_avg_graph = control_avg_graph / np.linalg.norm(control_avg_graph, 'fro') # normalizing it

case_avg_vector = np.mean(vectors[23:,:], axis=0)
case_avg_graph = np.zeros(shape=(n_labels, n_labels))
case_avg_graph[up_tri_idxs] = case_avg_vector
case_avg_graph = case_avg_graph + case_avg_graph.T
case_avg_graph = case_avg_graph / np.linalg.norm(case_avg_graph, 'fro') # normalizing it

diff_graph = control_avg_graph - case_avg_graph

labels = mne.read_labels_from_annot(subject='fsaverage', parc=atlas, subjects_dir=None, verbose=False)
if atlas == 'aparc.a2009s':
    labels = labels[:-2]
if atlas == 'aparc':
    labels = labels[:-1]

node_coords = []
for label in labels:
    if label.hemi == 'lh':
        hemi = 0
    if label.hemi == 'rh':
        hemi = 1
    center_vertex = label.center_of_mass(subject='fsaverage', 
                                        restrict_vertices=False, 
                                        subjects_dir=None)
    mni_pos = mne.vertex_to_mni(center_vertex, hemis=hemi,
                            subject='fsaverage', subjects_dir=None)
    node_coords.append(mni_pos)

node_coords = np.array(node_coords)
ticks = [lb.name for lb in labels]

diff_graph = control_avg_graph - case_avg_graph
zero_matrix = np.zeros(shape=(n_labels, n_labels))
for i, j in my_stat_dict["bonferroni"]:
    zero_matrix[i][j] = diff_graph[i][j]
stat_graph = zero_matrix + zero_matrix.T  

fig, ax = plt.subplots(1, 1, figsize=(11, 5))
plot_connectome(adjacency_matrix=stat_graph, node_coords=node_coords, display_mode="lzry",
                node_color='k', node_size=20, axes=ax, colorbar=False,
                edge_threshold="90%")
fig.tight_layout()



Comparison between averages of groups and one per group

In [ ]:
normalize = "demean-2norm"
sfreq = 250
end_sample = 5 * 60 * sfreq 
atlas = 'aparc'
freq_range = 'alpha'
method = 'mean'
dirs = ['/Users/payamsadeghishabestari/meg_gsp/stc_labels_morphed/tinmeg1',
        '/Users/payamsadeghishabestari/meg_gsp/stc_labels_morphed/tinmeg3']
subjects_dir = '/Applications/freesurfer/7.4.1/subjects'
fnames = []
for dir in dirs:
    for filename in sorted(os.listdir(dir)): 
        f = os.path.join(dir, filename)
        if os.path.isfile(f) and f.endswith(f'{atlas}_{method}.npy') and freq_range in f:
            fnames.append(f)

print(f'Computing the Graph Matrixes ...')
if atlas == 'aparc':
    n_labels = 68
if atlas == 'aparc.a2009s':
    n_labels = 148

# average of case and control subjects
graphs = []
for fname in tqdm(fnames):
    if atlas == 'aparc':
        stc_in_labels = np.load(file=fname, allow_pickle=True)[:-1,:]
    if atlas == 'aparc.a2009s':
        stc_in_labels = np.load(file=fname, allow_pickle=True)

    if normalize == "z-scoring":
        stc_in_labels = (stc_in_labels - np.mean(stc_in_labels, axis=0)) / np.std(stc_in_labels, axis=0)
    
    if normalize == "demean-2norm":
        mean_per_label = np.mean(stc_in_labels, axis=1, keepdims=True)
        norm_per_label = np.linalg.norm(stc_in_labels - mean_per_label, ord=2, axis=0)
        norm_per_label[norm_per_label == 0] = 1  
        stc_in_labels = (stc_in_labels - mean_per_label) / norm_per_label

    graph = learn_graph.log_degree_barrier(X=stc_in_labels[:,:end_sample], dist_type='sqeuclidean',
                                        alpha=1, beta=1, step=0.5, w0=None, maxit=10000,
                                        rtol=1e-16, retall=False, verbosity='NONE')
    graph = graph / np.linalg.norm(graph, 'fro') # normalizing it
    graphs.append(graph)
    
graphs = np.array(graphs)
avg_graph_control = graphs[:23,:,:].mean(axis=0)
avg_graph_case = graphs[23:,:,:].mean(axis=0)

# one graph for all case and all control subjects 
stcs = []
for fname in tqdm(fnames):
    if atlas == 'aparc':
        stc_in_labels = np.load(file=fname, allow_pickle=True)[:-1,:]
    if atlas == 'aparc.a2009s':
        stc_in_labels = np.load(file=fname, allow_pickle=True)

    stcs.append(stc_in_labels[:,:end_sample])

x_control = np.concatenate(stcs[:23], axis=1)
x_case = np.concatenate(stcs[23:], axis=1)

graph_control = learn_graph.log_degree_barrier(X=x_control, dist_type='sqeuclidean',
                                        alpha=1, beta=1, step=0.5, w0=None, maxit=10000,
                                        rtol=1e-16, retall=False, verbosity='NONE')
graph_control = graph_control / np.linalg.norm(graph_control, 'fro') # normalizing it

graph_case = learn_graph.log_degree_barrier(X=x_case, dist_type='sqeuclidean',
                                        alpha=1, beta=1, step=0.5, w0=None, maxit=10000,
                                        rtol=1e-16, retall=False, verbosity='NONE')
graph_case = graph_case / np.linalg.norm(graph_case, 'fro') # normalizing it


Visualizing

In [ ]:
if atlas == 'aparc':
    labels = mne.read_labels_from_annot(subject="fsaverage", parc=atlas,
                                        subjects_dir=subjects_dir, verbose=False)[:-1]
if atlas == 'aparc.a2009s':
    labels = mne.read_labels_from_annot(subject="fsaverage", parc=atlas,
                                        subjects_dir=subjects_dir, verbose=False)[:-2]
node_coords = []
for label in labels:
    if label.hemi == 'lh':
        hemi = 0
    if label.hemi == 'rh':
        hemi = 1
    center_vertex = label.center_of_mass(subject="fsaverage", 
                                        restrict_vertices=False, 
                                        subjects_dir=subjects_dir)
    mni_pos = mne.vertex_to_mni(center_vertex, hemis=hemi,
                            subject="fsaverage", subjects_dir=subjects_dir)
    node_coords.append(mni_pos)

node_coords = np.array(node_coords)
ticks = [lb.name for lb in labels]

fig, axes = plt.subplots(2, 2, figsize=(15, 7))
plot_connectome(adjacency_matrix=avg_graph_control, node_coords=node_coords,
                node_color='k', node_size=10, axes=axes[0][0],
                edge_threshold='93%')

plot_connectome(adjacency_matrix=avg_graph_case, node_coords=node_coords,
                node_color='k', node_size=10, axes=axes[1][0],
                edge_threshold='93%')

plot_connectome(adjacency_matrix=graph_control, node_coords=node_coords,
                node_color='k', node_size=10, axes=axes[0][1],
                edge_threshold='93%')

plot_connectome(adjacency_matrix=graph_case, node_coords=node_coords,
                node_color='k', node_size=10, axes=axes[1][1],
                edge_threshold='93%')

axes[0][0].set_ylabel("Control", fontsize=20)
axes[1][0].set_ylabel("Case", fontsize=20)
axes[0][0].set_title("Average of Group", fontsize=17)
axes[0][1].set_title("One Graph per Group", fontsize=17)

UMAP (both alpha and gamma)

In [ ]:
# loading subjects
cmap = plt.get_cmap("Set1")
atlas = 'aparc'
method = 'mean'
dirs = ['/Users/payamsadeghishabestari/meg_gsp/stc_labels_morphed/tinmeg1',
        '/Users/payamsadeghishabestari/meg_gsp/stc_labels_morphed/tinmeg3']

fnames_alpha = []; fnames_gamma = []
for dir in dirs:
    for filename in sorted(os.listdir(dir)): 
        f = os.path.join(dir, filename)
        if os.path.isfile(f) and f.endswith(f'{atlas}_{method}.npy'):
            if "alpha" in f:
                fnames_alpha.append(f)
            if "gamma" in f:
                fnames_gamma.append(f)

# creating vectors for each time period
print(f'Computing the Graph Matrixes ...')
sfreq = 250
init_time = 2
end_time = 300
n_frames = 100
if atlas == 'aparc':
    n_labels = 68
if atlas == 'aparc.a2009s':
    n_labels = 148
up_tri_idxs = np.triu_indices(n_labels, k=1)
durations = (np.linspace(init_time, end_time, n_frames) * sfreq).astype(int)
all_vectors_alpha = []; all_vectors_gamma = []

for fname_alpha, fname_gamma in tzip(fnames_alpha, fnames_gamma):
    vectors_alpha = np.zeros(shape=(len(durations), len(up_tri_idxs[0])))
    vectors_gamma = np.zeros(shape=(len(durations), len(up_tri_idxs[0])))

    stc_in_labels_alpha = np.load(file=fname_alpha, allow_pickle=True)
    stc_in_labels_gamma = np.load(file=fname_gamma, allow_pickle=True)
    
    if atlas == "aparc":
        stc_in_labels_alpha = stc_in_labels_alpha[:-1,:]
        stc_in_labels_gamma = stc_in_labels_gamma[:-1,:]
    
    for idx, duration in enumerate(durations):
        graph_alpha = learn_graph.log_degree_barrier(X=stc_in_labels_alpha[:, :duration],
                                                dist_type='sqeuclidean', alpha=1,
                                                beta=1, step=0.5, w0=None, maxit=10000,
                                                rtol=1e-16, retall=False, verbosity='NONE')
        graph_gamma = learn_graph.log_degree_barrier(X=stc_in_labels_gamma[:, :duration],
                                                dist_type='sqeuclidean', alpha=1,
                                                beta=1, step=0.5, w0=None, maxit=10000,
                                                rtol=1e-16, retall=False, verbosity='NONE')
        
        
        vectors_alpha[idx] = graph_alpha[up_tri_idxs]
        vectors_gamma[idx] = graph_gamma[up_tri_idxs]
    
    all_vectors_alpha.append(vectors_alpha)
    all_vectors_gamma.append(vectors_gamma)

all_vectors_alpha = np.array(all_vectors_alpha)
all_vectors_gamma = np.array(all_vectors_gamma)

all_vectors = np.concatenate((all_vectors_alpha, all_vectors_gamma), axis=0)

# going to umap space
print(f'Creating the UMAP space data ...')
data_transformed = []
for i in tqdm(range(n_frames)):
    data = torch.tensor(all_vectors[:,i,:], dtype=torch.float32)
    reshaped_data = data.reshape(data.shape[0], -1)
    umap_transform = UMAP(n_components=2, n_neighbors=3, 
                    learning_rate = 1e-7, metric='euclidean')
    data_transformed.append(umap_transform.fit_transform(reshaped_data))

# creating the animation
colors = [cmap.colors[0]] * 23 + [cmap.colors[1]] * 18 + [cmap.colors[2]] * 23 + [cmap.colors[3]] * 18 
fig, ax = plt.subplots()
ax.set_xlim(-2, 12)
ax.set_ylim(-2, 12)
ax.grid()
ax.set_xlabel('UMAP componet 1')
ax.set_ylabel('UMAP componet 2')
scatter = ax.scatter([], [], s=25)
text = ax.text(-1.5, -1.5, '', fontsize='large', fontstyle='italic')
def update(frame):
    x = data_transformed[frame][:, 0]
    y = data_transformed[frame][:, 1]
    offsets = np.column_stack((x, y))
    scatter.set_offsets(offsets)
    scatter.set_color(colors)
    text.set_text(f'Frame {frame}/{n_frames}')
    return scatter,

animation = FuncAnimation(fig, update, frames=n_frames, init_func=None, repeat=False, interval=300)
plt.show()

# saving the animation
animation.save(f"/Users/payamsadeghishabestari/meg_gsp/{atlas}_{method}_umap_morphed.gif", writer="pillow")
plt.close(animation._fig)


Bootstrapping

In [ ]:
# initiate parameters
F = 100
sfreq = 250
end_sample = 5 * 60 * sfreq 
atlas = 'aparc'
freq_range = 'alpha'
method = 'mean'
subjects_dir = '/Applications/freesurfer/7.4.1/subjects'
dirs = ['/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1',
        '/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg3']
if atlas == 'aparc':
    n_labels = 68
if atlas == 'aparc.a2009s':
    n_labels = 148

# concatenate controls and cases
fnames = []
for dir in dirs:
    for filename in sorted(os.listdir(dir)): 
        f = os.path.join(dir, filename)
        if os.path.isfile(f) and f.endswith(f'{atlas}_{method}.npy') and freq_range in f:
            fnames.append(f)
stcs = []
for fname in fnames:
    stc_in_labels = np.load(file=fname, allow_pickle=True)
    stcs.append(stc_in_labels[:,:end_sample])
x_control = np.concatenate(stcs[:23], axis=1)
x_case = np.concatenate(stcs[23:], axis=1)

n_surrogates = 100
up_tri_idxs = np.triu_indices(n_labels, k=1)
vectors_control = np.zeros(shape=(n_surrogates, len(up_tri_idxs[0])))
vectors_case = np.zeros(shape=(n_surrogates, len(up_tri_idxs[0])))

n_samples = F * sfreq # fixed to 100
    
# loop over N surrogates
for surr_idx in tqdm(range(n_surrogates)):
    
    # generate random indexes
    idxs_control = np.random.randint(1, x_control.shape[1], size=n_samples)
    idxs_case = np.random.randint(1, x_case.shape[1], size=n_samples)

    # construct graphs per control and case
    graph_control = learn_graph.log_degree_barrier(X=x_control[:, idxs_control],
                                            dist_type='sqeuclidean', alpha=1,
                                            beta=1, step=0.5, w0=None, maxit=10000,
                                            rtol=1e-16, retall=False, verbosity='NONE')
    graph_case = learn_graph.log_degree_barrier(X=x_case[:, idxs_case],
                                            dist_type='sqeuclidean', alpha=1,
                                            beta=1, step=0.5, w0=None, maxit=10000,
                                            rtol=1e-16, retall=False, verbosity='NONE')
    
    # normalizing graphs
    graph_control = graph_control / np.linalg.norm(graph_control, 'fro') 
    graph_case = graph_case / np.linalg.norm(graph_case, 'fro')

    vectors_control[surr_idx] = graph_control[up_tri_idxs]
    vectors_case[surr_idx] = graph_case[up_tri_idxs]
    
# removing very small connections
for i in range(n_surrogates):
    zero_edges = np.where(vectors_control[i] < vectors_control[i].max() * 0.01)[0]
    vectors_control[i][zero_edges] = 0
    zero_edges = np.where(vectors_case[i] < vectors_case[i].max() * 0.01)[0]
    vectors_case[i][zero_edges] = 0


In [ ]:
# statistics 1
(p_thr, alpha) = (0.05, 0.05)  
stat, p_values = ttest_ind(vectors_control, vectors_case, permutations=0, random_state=None)
my_stat_dict = {}
pvals_dict = {}
methods = ['bonferroni', 'sidak', 'holm-sidak', 'holm', 'simes-hochberg',
            'hommel', 'fdr_bh', 'fdr_by', 'fdr_tsbh', 'fdr_tsbky']
for method in methods:
    reject_null, p_corrected, _, _ = sm.stats.multipletests(pvals=p_values, alpha=alpha,
                                                        method=method)
    label_idxs = []
    if len(np.where(p_corrected < p_thr)[0]) == 0:
        print(f"No statistical difference between brain labels for method {method}")
    else:
        for idx in np.where(p_corrected < p_thr)[0]:
            label_idxs.append((up_tri_idxs[0][idx], up_tri_idxs[1][idx]))
    my_stat_dict[method] = label_idxs
    pvals_dict[method] = p_corrected

In [ ]:
# statistics 2
(p_thr, alpha) = (0.001, 0.001)  
stat, p_values = ttest_ind(vectors_control, vectors_case, permutations=0, alternative="two-sided", random_state=None)
# Benjamini/Hochberg correction
reject_null, p_corrected, _, _ = sm.stats.multipletests(pvals=p_values, alpha=alpha,
                                                        method='bonferroni')
label_idxs1 = []
label_idxs2 = []
k1 = int(len(p_corrected) * 0.01)
k2 = int(len(p_corrected) * 0.05)
for idx in np.argpartition(p_corrected, k1)[:k1]:
    label_idxs1.append((up_tri_idxs[0][idx], up_tri_idxs[1][idx]))
for idx in np.argpartition(p_corrected, k2)[:k2]:
    label_idxs2.append((up_tri_idxs[0][idx], up_tri_idxs[1][idx]))

In [ ]:
# visualization 1
atlas = "aparc"
labels = mne.read_labels_from_annot(subject='fsaverage', parc=atlas, subjects_dir=None, verbose=False)
if atlas == 'aparc.a2009s':
    labels = labels[:-2]
if atlas == 'aparc':
    labels = labels[:-1]

node_coords = []
for label in labels:
    if label.hemi == 'lh':
        hemi = 0
    if label.hemi == 'rh':
        hemi = 1
    center_vertex = label.center_of_mass(subject='fsaverage', 
                                        restrict_vertices=False, 
                                        subjects_dir=None)
    mni_pos = mne.vertex_to_mni(center_vertex, hemis=hemi,
                            subject='fsaverage', subjects_dir=None)
    node_coords.append(mni_pos)

node_coords = np.array(node_coords)
ticks = [lb.name for lb in labels]

n = 5
fig, axs = plt.subplots(2, n, figsize=(11, 5))
for i, j in zip(range(n), np.random.randint(1, n_surrogates, size=n)):
    matrix = np.zeros(shape=(n_labels, n_labels))
    matrix[up_tri_idxs] = vectors_control[j]
    plot_connectome(adjacency_matrix=matrix + matrix.T, node_coords=node_coords, display_mode="z",
                    node_color='k', node_size=20, axes=axs[0][i], colorbar=False,
                    edge_threshold="95%")

for i, j in zip(range(n), np.random.randint(1, n_surrogates, size=n)):
    matrix = np.zeros(shape=(n_labels, n_labels))
    matrix[up_tri_idxs] = vectors_case[j]
    plot_connectome(adjacency_matrix=matrix + matrix.T, node_coords=node_coords, display_mode="z",
                    node_color='k', node_size=20, axes=axs[1][i], colorbar=False,
                    edge_threshold="95%")


# fig.tight_layout()
axs[0][2].set_title(f"Frame size used: {int(n_samples/sfreq)} seconds")

In [ ]:
# visualization 2
control_avg_vector = np.mean(vectors_control, axis=0)
control_avg_graph = np.zeros(shape=(n_labels, n_labels))
control_avg_graph[up_tri_idxs] = control_avg_vector
control_avg_graph = control_avg_graph + control_avg_graph.T


case_avg_vector = np.mean(vectors_case, axis=0)
case_avg_graph = np.zeros(shape=(n_labels, n_labels))
case_avg_graph[up_tri_idxs] = case_avg_vector
case_avg_graph = case_avg_graph + case_avg_graph.T

diff_graph = control_avg_graph - case_avg_graph

labels = mne.read_labels_from_annot(subject='fsaverage', parc=atlas, subjects_dir=None, verbose=False)
if atlas == 'aparc.a2009s':
    labels = labels[:-2]
if atlas == 'aparc':
    labels = labels[:-1]

node_coords = []
for label in labels:
    if label.hemi == 'lh':
        hemi = 0
    if label.hemi == 'rh':
        hemi = 1
    center_vertex = label.center_of_mass(subject='fsaverage', 
                                        restrict_vertices=False, 
                                        subjects_dir=None)
    mni_pos = mne.vertex_to_mni(center_vertex, hemis=hemi,
                            subject='fsaverage', subjects_dir=None)
    node_coords.append(mni_pos)

node_coords = np.array(node_coords)
ticks = [lb.name for lb in labels]

diff_graph = control_avg_graph - case_avg_graph
zero_matrix = np.zeros(shape=(n_labels, n_labels))
for i, j in label_idxs1:
    zero_matrix[i][j] = diff_graph[i][j]
stat_graph1 = zero_matrix + zero_matrix.T  

zero_matrix = np.zeros(shape=(n_labels, n_labels))
for i, j in label_idxs2:
    zero_matrix[i][j] = diff_graph[i][j]
stat_graph2 = zero_matrix + zero_matrix.T  

fig, axs = plt.subplots(2, 1, figsize=(11, 5))
plot_connectome(adjacency_matrix=stat_graph1, node_coords=node_coords, display_mode="lzry",
                node_color='k', node_size=20, axes=axs[0], colorbar=False,
                edge_threshold="10%")
plot_connectome(adjacency_matrix=stat_graph2, node_coords=node_coords, display_mode="lzry",
                node_color='k', node_size=20, axes=axs[1], colorbar=False,
                edge_threshold="10%")
fig.tight_layout()

Compute Phase Amplitude Coupling (run once for all subjects)

In [ ]:
directory = '/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg3/alpha' 
subject_ids = []
for file in sorted(os.listdir(directory)): ## iterate over folders in that directory
    f = os.path.join(directory, file)
    if f.endswith(".fif"): ## select only folders
        subject_ids.append(f[-17:-13])

In [ ]:
directory = '/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg3/alpha' 
subject_ids = []
for file in sorted(os.listdir(directory)): ## iterate over folders in that directory
    f = os.path.join(directory, file)
    if f.endswith(".fif"): ## select only folders
        subject_ids.append(f[-17:-13])

pacs_dict = {"ndPAC": []}
idpac = (4, 0, 0)
n_labels = 68

for subject_id in subject_ids[3:]:
    
    # read data
    fname_alpha = f'/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg3/subject_{subject_id}_alpha_aparc_mean.npy'
    fname_gamma = f'/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg3/subject_{subject_id}_gamma_aparc_mean.npy'
    stc_alpha = np.load(file=fname_alpha, allow_pickle=True)
    stc_gamma = np.load(file=fname_gamma, allow_pickle=True)
    
    # reshape them
    data_pha = np.array([stc_alpha])
    data_amp = np.array([stc_gamma])

    # compute pac
    pacs = np.zeros(shape=(n_labels, n_labels))
    for ch_idx1, ch_idx2 in product(range(n_labels), range(n_labels)):
        pac_obj = Pac(idpac=idpac, f_pha=[8, 13], f_amp=[30, 80], verbose=None)
        pac = pac_obj.filterfit(sf=250, x_pha=data_pha[:,ch_idx1,:], x_amp=data_amp[:,ch_idx2,:], n_perm=200, p=0.05, mcp="bonferroni")
        pacs[ch_idx1][ch_idx2] = np.squeeze(pac)
    pacs_dict["ndPAC"] = pacs

    # save
    with open(f'/Users/payamsadeghishabestari/meg_gsp/pacs/alpha-gamma/case/{subject_id}.pkl', 'wb') as file:
        pickle.dump(pacs_dict, file)

Make two layer graph

In [ ]:
subject_id = "539"
fs = 250
n_labels = 68
tril_idxs = np.tril_indices(2*n_labels, k=-1)

# loading subject
fname_alpha = f'/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1/subject_{subject_id}_alpha_aparc_mean.npy'
fname_gamma = f'/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1/subject_{subject_id}_gamma_aparc_mean.npy'
pac_file_path = f'/Users/payamsadeghishabestari/meg_gsp/pacs/alpha-gamma/control/{subject_id}.pkl'
stc_alpha = np.load(file=fname_alpha, allow_pickle=True)
stc_gamma = np.load(file=fname_gamma, allow_pickle=True)
with open(pac_file_path, 'rb') as file:
    pac_dict = pickle.load(file)

# creating the z matrix
z = np.zeros(shape=(2 * n_labels, 2 * n_labels))
z1 = np.zeros(shape=(n_labels, n_labels))
z2 = pac_dict["ndPAC"]
z3 = z2.T
z4 = np.zeros(shape=(n_labels, n_labels))

for idx1, idx2 in product(range(n_labels), range(n_labels)):
    z1[idx1][idx2] = coherence(x=stc_alpha[idx1], y=stc_alpha[idx2], fs=fs)[1].mean()
    z4[idx1][idx2] = coherence(x=stc_gamma[idx1], y=stc_gamma[idx2], fs=fs)[1].mean()

z[:n_labels, :n_labels] = z1
z[:n_labels, n_labels:2*n_labels] = z2
z[n_labels:2*n_labels, :n_labels] = z3
z[n_labels:2*n_labels, n_labels:2*n_labels] = z4
z[np.isnan(z)] = 0
z = 1 - z[tril_idxs] # 1-coh

# graph learning
graph = learn_graph.two_layer_log_degree_barrier(z=z, alpha=1, beta=1, step=0.5,
                                        w0=None, maxit=10000, rtol=1e-16,
                                        retall=False, verbosity='NONE')
graph_alpha = graph[:n_labels, :n_labels]
graph_cfc = graph[:n_labels, n_labels:2*n_labels]
graph_gamma = graph[n_labels:2*n_labels, n_labels:2*n_labels]

## remove small connections at each graph separately
# thr = 0.01
# indices = graph_alpha < graph_alpha.max() * thr
# graph_alpha[indices] = 0
# indices = graph_cfc < graph_cfc.max() * thr
# graph_cfc[indices] = 0
# indices = graph_gamma < graph_gamma.max() * thr
# graph_gamma[indices] = 0

In [ ]:
# visualizing
graph_type = "cfc"
subjects_dir = '/Applications/freesurfer/7.4.1/subjects'
labels = mne.read_labels_from_annot(subject="fsaverage", parc="aparc",
                                    subjects_dir=subjects_dir, verbose=False)[:-1]
node_coords = []
for label in labels:
    if label.hemi == 'lh':
        hemi = 0
    if label.hemi == 'rh':
        hemi = 1
    center_vertex = label.center_of_mass(subject="fsaverage", 
                                        restrict_vertices=False, 
                                        subjects_dir=None)
    mni_pos = mne.vertex_to_mni(center_vertex, hemis=hemi,
                            subject="fsaverage", subjects_dir=None)
    node_coords.append(mni_pos)

node_coords = np.array(node_coords)
ticks = [lb.name for lb in labels]

fig, axes = plt.subplots(1, 1, figsize=(11, 5))

if graph_type == "alpha":
    edge_kwargs = None
    sub_graph = graph_alpha
if graph_type == "gamma":
    edge_kwargs = None
    sub_graph = graph_gamma
if graph_type == "cfc":
    edge_kwargs = {"lw": 0.01}
    sub_graph = graph_cfc

plot_connectome(adjacency_matrix=sub_graph, node_coords=node_coords, display_mode="lzry",
                node_color='k', node_size=20, axes=axes, edge_threshold='99.8%', edge_kwargs=edge_kwargs)
fig.tight_layout()

Compare all subjects

In [ ]:
directory = '/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg1/alpha' 
subject_ids = []
for file in sorted(os.listdir(directory)): ## iterate over folders in that directory
    f = os.path.join(directory, file)
    if f.endswith(".fif"): ## select only folders
        subject_ids.append(f[-16:-13])

In [ ]:
## run it once
control_subjects_graph = {"alpha_graphs" : [], "gamma_graphs": [], "cfc_graphs": []}
fs = 250
n_labels = 68
tril_idxs = np.tril_indices(2*n_labels, k=-1)

for subject_id in tqdm(subject_ids):

    # loading subject
    fname_alpha = f'/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1/subject_{subject_id}_alpha_aparc_mean.npy'
    fname_gamma = f'/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1/subject_{subject_id}_gamma_aparc_mean.npy'
    pac_file_path = f'/Users/payamsadeghishabestari/meg_gsp/pacs/alpha-gamma/control/{subject_id}.pkl'
    stc_alpha = np.load(file=fname_alpha, allow_pickle=True)
    stc_gamma = np.load(file=fname_gamma, allow_pickle=True)
    with open(pac_file_path, 'rb') as file:
        pac_dict = pickle.load(file)

    # creating the z matrix
    z = np.zeros(shape=(2 * n_labels, 2 * n_labels))
    z1 = np.zeros(shape=(n_labels, n_labels))
    z2 = pac_dict["ndPAC"]
    z3 = z2.T
    z4 = np.zeros(shape=(n_labels, n_labels))

    for idx1, idx2 in product(range(n_labels), range(n_labels)):
        z1[idx1][idx2] = coherence(x=stc_alpha[idx1], y=stc_alpha[idx2], fs=fs)[1].mean()
        z4[idx1][idx2] = coherence(x=stc_gamma[idx1], y=stc_gamma[idx2], fs=fs)[1].mean()

    z[:n_labels, :n_labels] = z1
    z[:n_labels, n_labels:2*n_labels] = z2
    z[n_labels:2*n_labels, :n_labels] = z3
    z[n_labels:2*n_labels, n_labels:2*n_labels] = z4
    

    # handling nan values, because there is no signal in some labels
    z[np.isnan(z)] = 0
    z = 1 - z[tril_idxs] # 1-coh

    # graph learning
    graph = learn_graph.two_layer_log_degree_barrier(z=z, alpha=1, beta=1, step=0.5,
                                            w0=None, maxit=10000, rtol=1e-16,
                                            retall=False, verbosity='NONE')
    graph_alpha = graph[:n_labels, :n_labels]
    graph_cfc = graph[:n_labels, n_labels:2*n_labels]
    graph_gamma = graph[n_labels:2*n_labels, n_labels:2*n_labels]

    ## remove small connections at each graph separately
    # thr = 0.01
    # indices = graph_alpha < graph_alpha.max() * thr
    # graph_alpha[indices] = 0
    # indices = graph_cfc < graph_cfc.max() * thr
    # graph_cfc[indices] = 0
    # indices = graph_gamma < graph_gamma.max() * thr
    # graph_gamma[indices] = 0

    ## append to dict
    control_subjects_graph["alpha_graphs"].append(graph_alpha)
    control_subjects_graph["cfc_graphs"].append(graph_cfc)
    control_subjects_graph["gamma_graphs"].append(graph_gamma)

# save
with open(f'/Users/payamsadeghishabestari/meg_gsp/pacs/two_layer_graph/control_subjects_graph.pkl', 'wb') as file:
    pickle.dump(control_subjects_graph, file)

In [ ]:
## all brain connectivity
key = "cfc_graphs"
control_file_path = "/Users/payamsadeghishabestari/meg_gsp/pacs/two_layer_graph/control_subjects_graph.pkl"
case_file_path = "/Users/payamsadeghishabestari/meg_gsp/pacs/two_layer_graph/case_subjects_graph.pkl"

with open(control_file_path, 'rb') as file:
    control_dict = pickle.load(file)
with open(case_file_path, 'rb') as file:
    case_dict = pickle.load(file)

tril_idxs = np.tril_indices(68, k=-1)
controls_vector = np.array([i[tril_idxs] for i in np.array(control_dict[key])])
case_vector = np.array([i[tril_idxs] for i in np.array(case_dict[key])])


(p_thr, alpha) = (0.05, 0.05)  
stat, p_values = ttest_ind(controls_vector, case_vector, permutations=0, random_state=None)
reject_null, p_corrected, _, _ = sm.stats.multipletests(pvals=p_values, alpha=alpha,
                                                        method='fdr_bh')
label_idxs = []
if len(np.where(p_values < p_thr)[0]) == 0:
    print(f"No statistical difference between brain labels")
else:
    for idx in np.where(p_values < p_thr)[0]:
        label_idxs.append((tril_idxs[0][idx], tril_idxs[1][idx]))


Does alpha power decrease in tinnitus?

In [ ]:
# check brain labels
directory = '/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg1/alpha' 
subject_ids = []
for file in sorted(os.listdir(directory)): ## iterate over folders in that directory
    f = os.path.join(directory, file)
    if f.endswith(".fif"): ## select only folders
        subject_ids.append(f[-16:-13])

control_alpha_powers = []
for subject_id in subject_ids:
    fname_alpha = f'/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1/subject_{subject_id}_alpha_aparc_mean.npy'
    stc_alpha = np.load(file=fname_alpha, allow_pickle=True)
    control_alpha_powers.append(np.mean(np.square(stc_alpha), axis=1))

directory = '/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg3/alpha' 
subject_ids = []
for file in sorted(os.listdir(directory)): ## iterate over folders in that directory
    f = os.path.join(directory, file)
    if f.endswith(".fif"): ## select only folders
        subject_ids.append(f[-17:-13])

case_alpha_powers = []
for subject_id in subject_ids:
    fname_alpha = f'/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg3/subject_{subject_id}_alpha_aparc_mean.npy'
    stc_alpha = np.load(file=fname_alpha, allow_pickle=True)
    case_alpha_powers.append(np.mean(np.square(stc_alpha), axis=1))

labels = mne.read_labels_from_annot(subject="fsaverage", parc="aparc",
                                    subjects_dir=None, verbose=False)[:-1]
ticks = [lb.name for lb in labels]
(p_thr, alpha) = (0.05, 0.05)  
stat, p_values = ttest_ind(np.array(control_alpha_powers), np.array(case_alpha_powers),
                            permutations=0, random_state=None)
reject_null, p_corrected, _, _ = sm.stats.multipletests(pvals=p_values, alpha=alpha,
                                                        method='fdr_bh')

if len(np.where(p_corrected < p_thr)[0]) == 0:
    print(f"No statistical difference")
else:
    for idx in np.where(p_corrected < p_thr)[0]:
        print(f"difference at brain label: {ticks[idx]}")

In [ ]:
# check topomaps
directory = '/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg1/alpha' 
subject_ids = []
for file in sorted(os.listdir(directory)): ## iterate over folders in that directory
    f = os.path.join(directory, file)
    if f.endswith(".fif"): ## select only folders
        subject_ids.append(f[-16:-13])

spectra_control = []
for subject_id in tqdm(subject_ids):
    raw = mne.io.read_raw_fif(fname=f"/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg1/alpha/{subject_id}_raw_tsss.fif", preload=True, verbose=False)
    spectra_control.append(raw.compute_psd(fmin=0.1, fmax=40, verbose=False))


directory = '/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg3/alpha' 
subject_ids = []
for file in sorted(os.listdir(directory)): ## iterate over folders in that directory
    f = os.path.join(directory, file)
    if f.endswith(".fif"): ## select only folders
        subject_ids.append(f[-17:-13])

spectra_case = []
for subject_id in tqdm(subject_ids):
    raw = mne.io.read_raw_fif(fname=f"/Users/payamsadeghishabestari/meg_gsp/raws/tinmeg3/alpha/{subject_id}_raw_tsss.fif", preload=True, verbose=False)
    spectra_case.append(raw.compute_psd(fmin=0.1, fmax=40, verbose=False))

total_control = [i.get_data() for i in spectra_control]
data_control = np.array(total_control).mean(axis=0)
total_case = [i.get_data() for i in spectra_case]
data_case = np.array(total_case).mean(axis=0)
data_diff = data_control - data_case

info = spectra_control[0].info
freqs = spectra_control[0].freqs
grand_spectrum_control = mne.time_frequency.SpectrumArray(data_control, info, freqs)
grand_spectrum_case = mne.time_frequency.SpectrumArray(data_case, info, freqs)
diff_spectrum = mne.time_frequency.SpectrumArray(data_diff, info, freqs)

bands = {'Alpha (8-13 Hz)': (8, 13)}
diff_spectrum.plot_topomap(bands=bands, vlim=(0, 1000))

(p_thr, alpha) = (0.05, 0.05)  
stat, p_values = ttest_ind(np.array(total_control).mean(axis=2), np.array(total_case).mean(axis=2),
                            permutations=0, random_state=None)
reject_null, p_corrected, _, _ = sm.stats.multipletests(pvals=p_values, alpha=alpha,
                                                        method='fdr_bh')

In [ ]:
total_control_norm = np.array([i / np.array(total_control).mean(axis=0) for i in np.array(total_control)])
total_case_norm = np.array([i / np.array(total_case).mean(axis=0) for i in np.array(total_case)])

In [ ]:
stat, p_values = ttest_ind(np.array(total_control_norm).mean(axis=2), np.array(total_case_norm).mean(axis=2),
                            permutations=0, random_state=None)
reject_null, p_corrected, _, _ = sm.stats.multipletests(pvals=p_values, alpha=alpha,
                                                        method='fdr_bh')

K sample testing via independent tests

In [ ]:
from hyppo.ksample import KSample
import pandas as pd
import warnings
pd.options.mode.chained_assignment = None

In [ ]:
## subjects to select
co_include = range(23)
ca_include = range(18)
method = "fdr_bh"
key = "cfc_graphs"

## load all subjects
control_file_path = "/Users/payamsadeghishabestari/meg_gsp/pacs/two_layer_graph/control_subjects_graph.pkl"
case_file_path = "/Users/payamsadeghishabestari/meg_gsp/pacs/two_layer_graph/case_subjects_graph.pkl"

with open(control_file_path, 'rb') as file:
    control_dict = pickle.load(file)
with open(case_file_path, 'rb') as file:
    case_dict = pickle.load(file)

samples_co = np.array(control_dict[key])[co_include]
samples_ca = np.array(case_dict[key])[ca_include]

In [ ]:
## subjects to select
co_include = range(23)
ca_include = range(18)
method = "fdr_bh"
key = "cfc_graphs"

## load all subjects
control_file_path = "/Users/payamsadeghishabestari/meg_gsp/pacs/two_layer_graph/control_subjects_graph.pkl"
case_file_path = "/Users/payamsadeghishabestari/meg_gsp/pacs/two_layer_graph/case_subjects_graph.pkl"

with open(control_file_path, 'rb') as file:
    control_dict = pickle.load(file)
with open(case_file_path, 'rb') as file:
    case_dict = pickle.load(file)

samples_co = np.array(control_dict[key])[co_include]
samples_ca = np.array(case_dict[key])[ca_include]

connectomes = [samples_co, samples_ca]

indices = zip(*np.triu_indices(68, 1))
edge_pvals = []
for roi_i, roi_j in indices:

    # Get the (i,j)-th edge for each connectome
    samples = [type[:, roi_i, roi_j] for type in connectomes]

    # Calculate the p-value for the (i,j)-th edge
    try:
        statistic, pvalue = KSample("Dcorr").test(*samples, reps=10000, workers=-1)
    except ValueError:
        # A ValueError is thrown when any of the samples have equal edge
        # weights (i.e. one of the inputs has 0 variance)
        statistic = np.nan
        pvalue = 1

    edge_pvals.append([roi_i, roi_j, statistic, pvalue])

# Convert the nested list to a dataframe
edge_pvals_df = pd.DataFrame(edge_pvals, columns=["ROI_1", "ROI_2", "stat", "pval"])

# multiple comparison
alpha = 0.05
ps = [item[3] for item in edge_pvals]
reject_null, corr_ps, _, _ = sm.stats.multipletests(pvals=ps, alpha=alpha,
                                                        method=method)
for p_idx, p in enumerate(corr_ps):
    edge_pvals[p_idx][3] = p

edge_pvals_df["corr_pval"] = corr_ps
edge_pvals_df["significant"] = (edge_pvals_df["corr_pval"] < alpha)

# Get the top 10 strongest signal edges
edge_pvals_df.sort_values(by="corr_pval", inplace=True, ignore_index=True)
edge_pvals_top = edge_pvals_df.head(10)

# Replace ROI indices with actual names
labels = mne.read_labels_from_annot(subject="fsaverage", parc="aparc",
                                    subjects_dir=None, verbose=False)[:-1]
ticks = [lb.name for lb in labels]
def lookup_roi_name(index):
    roi_name = ticks[index]
    return f"{roi_name}"

edge_pvals_top["ROI_1"] = edge_pvals_top["ROI_1"].apply(lookup_roi_name)
edge_pvals_top["ROI_2"] = edge_pvals_top["ROI_2"].apply(lookup_roi_name)
edge_pvals_top.head()

Graph Matching

In [ ]:
import random
import networkx as nx
from graspologic.match import graph_match
from graspologic.utils import import_graph
import pandas as pd
from scipy.spatial import distance

In [ ]:
# create random indexes (sampling)
atlas = 'aparc'
freq_range = 'alpha'
method = 'mean'
sfreq = 250
n_labels = 68
init_time = 2
end_time = 300
up_tri_idxs = np.triu_indices(n_labels, k=1)
dirs = ['/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1',
        '/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg3']
fnames = []
for dir in dirs:
    for filename in sorted(os.listdir(dir)): 
        f = os.path.join(dir, filename)
        if os.path.isfile(f) and f.endswith(f'{atlas}_{method}.npy') and freq_range in f:
            fnames.append(f)

n_frames = 30
random_arrays = []
for n_sec in np.linspace(init_time, end_time, n_frames):
    random_array = random.sample(range(end_time*sfreq), k=int(n_sec*sfreq))
    random_arrays.append(random_array)

In [ ]:
# create dataframe
fname = "/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1/subject_539_gamma_aparc_mean.npy"
times = np.linspace(init_time, end_time, n_frames)
graph_df = {"subject_id": [], "frequency_range": [], "alpha": [], "beta": [], "modularity": [], 
            "avg_deg": [], "avg_cc": [], "coverage": [],
            "performance": [], "density": [], "euc_dist": [],
            "matching_score": [], "times_used": []}

stc_in_labels = np.load(file=fname, allow_pickle=True)
params = [0.1, 0.25, 0.5, 1, 1.5, 2, 2.5]


for idx, random_array in tqdm(enumerate(random_arrays)):    

    for alpha, beta in product(params, params): # loop over parameters    
        
        # learn graph
        graph = learn_graph.log_degree_barrier(X=stc_in_labels[:, random_array],
                                                dist_type='sqeuclidean', alpha=alpha,
                                                beta=beta, step=0.5, w0=None, maxit=10000,
                                                rtol=1e-16, retall=False, verbosity='NONE')
        # normalizing it
        graph = graph / np.linalg.norm(graph, 'fro')
        # drop small edges
        thr = 0.01 * np.max(graph)
        graph[graph < thr] = 0
        
        # compute preferences
        G = nx.from_numpy_matrix(np.array(graph))
        part = nx.community.greedy_modularity_communities(G)
        
        # fill the dataframe
        graph_df["modularity"].append(nx.community.modularity(G, part))
        graph_df["avg_deg"].append(np.mean([d for n, d in G.degree()]))
        graph_df["avg_cc"].append(nx.average_clustering(G))
        graph_df["coverage"].append(nx.community.partition_quality(G, part)[0])
        graph_df["performance"].append(nx.community.partition_quality(G, part)[1])
        graph_df["density"].append(nx.density(G))
        # graph_df["avg_short_path"].append(nx.average_shortest_path_length(G))

        # compute best graph
        graph_last = learn_graph.log_degree_barrier(X=stc_in_labels[:, :],
                                                dist_type='sqeuclidean', alpha=alpha,
                                                beta=beta, step=0.5, w0=None, maxit=10000,
                                                rtol=1e-16, retall=False, verbosity='NONE')
        # normalizing it
        graph_last = graph_last / np.linalg.norm(graph_last, 'fro')
        # drop small edges
        thr = 0.01 * np.max(graph_last)
        graph_last[graph_last < thr] = 0

        # compute euclidean distance
        graph_df["euc_dist"].append(distance.euclidean(graph.flatten(), graph_last.flatten()))

        # graph matching
        G = import_graph(G)
        G_last = nx.from_numpy_matrix(np.array(graph_last))
        G_last = import_graph(G_last)
        graph_df["matching_score"].append(graph_match(G, G_last)[2])

        # fill the parameters
        graph_df["alpha"].append(alpha)
        graph_df["beta"].append(beta)
        graph_df["times_used"].append(times[idx])
        graph_df["subject_id"].append(fname[-24:-21])
        graph_df["frequency_range"].append("gamma")


df = pd.DataFrame(graph_df)
df.to_csv("/Users/payamsadeghishabestari/meg_gsp/graph_matching/df_10.csv")

In [ ]:
# load and plot the dataframes
dfs_list = [pd.read_csv(f"/Users/payamsadeghishabestari/meg_gsp/graph_matching/df_{i}.csv") for i in range(1, 11)]
df = pd.concat(dfs_list)

mask1 = df["alpha"].isin([0.1, 0.25, 0.5, 1, 1.5, 2, 2.5])
df = df[mask1]
mask2 = df["beta"].isin([0.5, 1, 1.5])
df = df[mask2]
df_alpha = df[df["frequency_range"]=="alpha"]
df_gamma = df[df["frequency_range"]=="gamma"]

In [ ]:
kwargs = {"markersize": 3, "lw": 2, "ax": ax}
g = sns.catplot(data=df_alpha, x="times_used", y="matching_score", hue="beta", col="alpha", # row="frequency_range",
    capsize=.2, palette="YlGnBu_d", errorbar="se",
    kind="point", markers=["o", "*", "s"], height=6, aspect=.75, **kwargs)
g.set_xticklabels([])

Comparing networks with Classical correlation methods

In [ ]:
from mne_connectivity import spectral_connectivity_time

In [ ]:
## parameters to adjust
statistics = "t-test" # "Mann-Whitney U test"
F = 300
sfreq = 250
alpha = 0.05  # significance level
p_thr = 0.05
end_sample = F * sfreq

# loading subjects
atlas = 'aparc'
freq_ranges = ["delta", "theta", "alpha", "beta", "gamma"]
method = 'mean'
dirs = ['/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1',
        '/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg3']

labels = mne.read_labels_from_annot(subject='fsaverage', parc=atlas, subjects_dir=None, verbose=False)[:-1]
lb_names = np.array([lb.name for lb in labels])

stats_dict = {}
pvals_dict = {}

freqs = np.linspace(30, 80, 6)
for freq_range in freq_ranges[4:]:    
    fnames = []
    for dir in dirs:
        for filename in sorted(os.listdir(dir)): 
            f = os.path.join(dir, filename)
            if os.path.isfile(f) and f.endswith(f'{atlas}_{method}.npy') and freq_range in f:
                fnames.append(f)

# creating co
print(f'Computing the Graph Matrixes ...')
if atlas == 'aparc':
    n_labels = 68
if atlas == 'aparc.a2009s':
    n_labels = 148
up_tri_idxs = np.triu_indices(n_labels, k=1)
vectors = []

for fname in tqdm(fnames):
    stc_in_labels = np.load(file=fname, allow_pickle=True)
    data = np.array([stc_in_labels])

    con = spectral_connectivity_time(data=data, freqs=freqs, method="wpli",
                                average=True, sfreq=sfreq, faverage=True,
                                verbose=False)
    con_data = con.get_data(output="dense")[:,:,0].T
    
    vector = con_data[up_tri_idxs]
    vectors.append(vector)

vectors = np.array(vectors)

# removing very small connections
# for i in range(len(fnames)):
#     zero_edges = np.where(vectors[i] < vectors[i].max() * 0.01)[0]
#     vectors[i][zero_edges] = 0

# statistics
control_group = vectors[:23,:]
case_group = vectors[23:,:]

control_group[np.where(np.isnan(control_group))] = 0
case_group[np.where(np.isnan(case_group))] = 0

if statistics == 't-test':
    stat, p_values = ttest_ind(control_group, case_group,
                                permutations=0, random_state=42)
if statistics == 'Mann-Whitney U test':
    stat, p_values = mannwhitneyu(control_group, case_group)

my_stat_dict = {}
p_dict = {}
methods = ['bonferroni', 'fdr_bh']
for method in methods:
    reject_null, p_corrected, _, _ = sm.stats.multipletests(pvals=p_values, alpha=alpha,
                                                        method=method)
    label_idxs = []
    if len(np.where(p_corrected < p_thr)[0]) == 0:
        print(f"No statistical difference between brain labels for method {method}")
    else:
        for idx in np.where(p_corrected < p_thr)[0]:
            label_idxs.append((up_tri_idxs[0][idx], up_tri_idxs[1][idx]))
    my_stat_dict[method] = lb_names[label_idxs]
    p_dict[method] = p_corrected

stats_dict[freq_range] = my_stat_dict
pvals_dict[freq_range] = p_dict

Check the labels thickness / volume / area and graph

In [ ]:
mode = "thickness"
hemisphere = "lh"
mc_method = 'bonferroni' # 'fdr_bh'
labels = mne.read_labels_from_annot(subject='fsaverage', parc="aparc", subjects_dir=None, verbose=False)[:-1]
lb_names = np.array([lb.name for lb in labels])

dfs_list = []
for hemi in ["lh", "rh"]:
    fname = f"/Users/payamsadeghishabestari/KI_MEG/structural_stats/aparc_stats_{mode}_{hemi}.csv"
    df = pd.read_csv(fname, sep="\s+")
    new_column_names = [col[3:] for col in df.columns]
    df.columns = new_column_names
    df["hemi"] = hemi
    group_ids = ["control"] * 26 + ["case"] * 18
    df["group"] = group_ids
    dfs_list.append(df)

df = pd.concat(dfs_list)
df = df.drop(columns=['inSegVolNotVent', 'V'])

if mode == "thickness":
    df = df.drop(columns="MeanThickness_thickness")
if mode == "area":
    df = df.drop(columns="WhiteSurfArea_area")   

mask = df[f"aparc.{mode}"].isin([697, 750, 853]) # drop these subjects
df = df[~mask]
df = df.reset_index().drop(columns="index")
df_hemi = df[df["hemi"]==f'{hemisphere}']
df_control = df_hemi.query('group=="control"')
df_case = df_hemi.query('group=="case"')

##
my_dict = {"label_name": [],
            "hemisphere": [],
            f"average_{mode}_control": [],
            f"average_{mode}_case": [],
            "p_value": [],
            "p_corrected": [],
            "significant": []}

p_values = []
for col in list(df_hemi.columns[1:-2]):
    stat, p_value = ttest_ind(df_control[col], df_case[col],
                                permutations=0, random_state=42)
    p_values.append(p_value)

    my_dict["label_name"].append(col)
    my_dict[f"average_{mode}_control"].append(df_control[col].mean())
    my_dict[f"average_{mode}_case"].append(df_case[col].mean())
    my_dict["p_value"].append(p_value)

## after multiple comparison
reject_null, p_corrected, _, _ = sm.stats.multipletests(pvals=p_values, alpha=0.05,
                                                        method=mc_method)

my_dict["p_corrected"] = p_corrected
my_dict["significant"] = reject_null
my_dict["hemisphere"] = hemisphere
df_stat = pd.DataFrame(my_dict)
df_stat.sort_values(by="p_corrected").head()

In [ ]:
## plotting the significant ones
y = "superiorparietal_thickness"
palette_color = ['#1f77b4', '#d62728']
fig, ax = plt.subplots(1, 1, figsize=(6, 3))
sns.violinplot(data=df, x="group", y=y, hue="hemi",
                palette=palette_color, fill=False, ax=ax, inner=None, legend=False)
sns.swarmplot(data=df, x="group", y=y, hue="hemi", dodge=True,
                palette=palette_color, ax=ax)
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)

In [ ]:
df.drop(columns="aparc.thickness", inplace=True)

## construct a graph for control
df_sub = df[df["group"]==f"control"]
corr_matrix_control = np.zeros(shape=(len(lb_names), len(lb_names)))

for lb1_idx, lb1 in enumerate(lb_names):
    for lb2_idx, lb2 in enumerate(lb_names):
        (hemi1, hemi2) = (lb1[-2:], lb2[-2:])

        df_sub1 = df_sub[df_sub["hemi"]==f"{hemi1}"]
        df_sub2 = df_sub[df_sub["hemi"]==f"{hemi2}"]

        x = df_sub1[f"{lb1[:-3]}_thickness"]
        y = df_sub2[f"{lb2[:-3]}_thickness"]
        corr_matrix_control[lb1_idx][lb2_idx] = np.corrcoef(x.values, y.values)[0][1]

## construct a graph for case
df_sub = df[df["group"]==f"case"]
corr_matrix_case = np.zeros(shape=(len(lb_names), len(lb_names)))

for lb1_idx, lb1 in enumerate(lb_names):
    for lb2_idx, lb2 in enumerate(lb_names):
        (hemi1, hemi2) = (lb1[-2:], lb2[-2:])

        df_sub1 = df_sub[df_sub["hemi"]==f"{hemi1}"]
        df_sub2 = df_sub[df_sub["hemi"]==f"{hemi2}"]

        x = df_sub1[f"{lb1[:-3]}_thickness"]
        y = df_sub2[f"{lb2[:-3]}_thickness"]
        corr_matrix_case[lb1_idx][lb2_idx] = np.corrcoef(x.values, y.values)[0][1]

## conver 1 to 0
for i, j in product(range(68), range(68)):
    if i == j:
        corr_matrix_control[i][j] = 0
        corr_matrix_case[i][j] = 0

## remove small connections
thr = 0.01 * corr_matrix_control.max()
corr_matrix_control[abs(corr_matrix_control) < thr] = 0

thr = 0.01 * corr_matrix_case.max()
corr_matrix_case[abs(corr_matrix_case) < thr] = 0

In [ ]:
subjects_dir = '/Applications/freesurfer/7.4.1/subjects'
labels = mne.read_labels_from_annot(subject="fsaverage", parc="aparc",
                                    subjects_dir=subjects_dir, verbose=False)[:-1]
node_coords = []
for label in labels:
    if label.hemi == 'lh':
        hemi = 0
    if label.hemi == 'rh':
        hemi = 1
    center_vertex = label.center_of_mass(subject="fsaverage", 
                                        restrict_vertices=False, 
                                        subjects_dir=None)
    mni_pos = mne.vertex_to_mni(center_vertex, hemis=hemi,
                            subject="fsaverage", subjects_dir=None)
    node_coords.append(mni_pos)

node_coords = np.array(node_coords)
ticks = [lb.name for lb in labels]
edge_kwargs = None

diff_corr = corr_matrix_case - corr_matrix_control
fig, axes = plt.subplots(1, 1, figsize=(11, 5))

plot_connectome(adjacency_matrix=diff_corr, node_coords=node_coords, display_mode="lzry",
                node_color='k', node_size=20, axes=axes, edge_threshold='99.7%', edge_kwargs=edge_kwargs)
fig.tight_layout()